In [1]:
# This is a cell to hide code snippets from displaying
# This must be at first cell!

from IPython.display import HTML

hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input style="opacity:0" type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
hide_me 

import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

# Graphen

## Inhalt
1. [Zielsetzungen](#Zielsetzungen)
2. [Vokabular und Definitionen](#VokabularUndDefinitionen)
3. [Der abstrakte Datentyp Graph](#abstrakteDatentyp)
4. [Die Adjazenzmatrix](#Adjazenzmatrix)
5. [Eine Adjazenzliste](#Adjazenzliste)
6. [Implementierung](#Implementierung)
7. [Die Wort-Leiter-Problematik](#Wort-Leiter-Problematik)
8. [Erstellen des Wort-Leiter-Graphen](#ErstellenDesWort-Leiter-Graphen)
9. [Implementierung von Breadth First Search](#ImplementierungBreadthFirstSearch)
10. [Analyse der Breadth First Suche](#AnalyseBreadthFirstSearch)
11. [Das Rittertour-Problem](#Rittertour-Problem)
12. [Den Graphen der Rittertour erstellen](#GraphenDerRittertour)
13. [Implementierung der Rittertour](#ImplementierungDerRittertour)
14. [Analyse der Rittertour](#AnalyseDerRittertour)
15. [Allgemeine Depth First Suche](#AllgemeineDepthFirstSuche)
16. [Analyse der Depth First Suche](#AnalyseDepthFirstSuche)
17. [Topologische Sortierung](#TopologischeSortierung)
18. [Stark verbundene Komponenten](#StarkVerbundeneKomponenten)
19. [Probleme mit dem kürzesten Weg](#ProblemeMitDemKürzestenWeg)
20. [Der Dijkstra Algorithmus](#DerDijkstraAlgorithmus)
21. [Analyse des Dijkstra Algorithmus](#AnalyseDijkstraAlgorithmus)
22. [Der A* Algorithmus](#DerA*Algorithmus)
23. [Prim's Spannbaum-Algorithmus](#PrimSpannbaum-Algorithmus)
24. [Zusammenfassung](#Zusammenfassung)
25. [Fragen zur Diskussion](#Diskussion)
26. [Programmieraufgaben](#Programmieraufgaben)

<a id='Zielsetzungen'></a>
## Zielsetzungen
* Erfahren, was ein Graph ist und wie er verwendet wird.
* Den abstrakten Datentyp des Graphen unter Verwendung mehrerer interner Darstellungen implementieren.
* Sehen, wie Graphen zur Lösung einer Vielzahl von Problemen verwendet werden können.

In diesem Kapitel werden wir Graphen untersuchen. Graphen sind eine allgemeinere Struktur als die Bäume, die wir im letzten Kapitel untersucht haben; tatsächlich kann man sich einen Baum als eine besondere Art von Graphen vorstellen. Graphen können verwendet werden, um viele interessante Dinge über unsere Welt darzustellen, wie z.B. Straßensysteme, Flugverbindungen von Stadt zu Stadt, wie das Internet angeschlossen ist oder sogar die Reihenfolge der Kurse, die Sie belegen müssen, um ein Hauptfach in Informatik zu absolvieren. Wir werden in diesem Kapitel sehen, dass wir, sobald wir eine gute Darstellung für ein Problem haben, einige Standardalgorithmen für Graphen verwenden können, um ein Problem zu lösen, das sonst sehr schwierig erscheinen würde.

Während es für Menschen relativ einfach ist, sich eine Straßenkarte anzuschauen und die Beziehungen zwischen verschiedenen Orten zu verstehen, verfügt ein Computer nicht über ein solches Wissen. Wir können uns eine Straßenkarte jedoch auch als Graph vorstellen. Wenn wir dies tun, können wir unseren Computer interessante Dinge für uns tun lassen. Wenn Sie schon einmal eine der Internet-Kartenseiten benutzt haben, wissen Sie, dass ein Computer den kürzesten, schnellsten oder einfachsten Weg von einem Ort zum anderen finden kann.

Als Informatikstudent fragen Sie sich vielleicht, welche Kurse Sie belegen müssen, um einen Abschluss zu erhalten. Ein Graph ist eine gute Möglichkeit, die Voraussetzungen und andere Abhängigkeiten zwischen den Studiengängen darzustellen. *Abbildung 1* zeigt einen weiteren Graphen. Dieses stellt die Kurse und die Reihenfolge dar, in der sie belegt werden müssen, um am Luther-Kolleg ein Hauptfach Informatik zu belegen.

<center><img src="Bilder/Graphen/graphen01.PNG"></center>
<center>Abbildung 1: Voraussetzungen für ein Informatik-Abschluss</center>


<a id='VokabularUndDefinitionen'></a>
## Vokabular und Definitionen

Nachdem wir uns nun einige Beispiele für Graphen angesehen haben, werden wir einen Graphen und seine Komponenten etwas formeller definieren. Einige dieser Begriffe kennen wir bereits aus unserer Diskussion über Bäume.

**Scheitelpunkt (Vertex)**
Ein Vertex (auch "Knoten" genannt) ist ein grundlegender Teil eines Graphen. Er kann einen Namen haben, den wir den "Schlüssel" nennen werden. Ein Vertex kann auch zusätzliche Informationen enthalten. Diese zusätzlichen Informationen werden wir "Nutzlast" nennen.

**Kante**
Eine Kante (auch "Bogen" genannt) ist ein weiterer grundlegender Teil eines Graphen. Eine Kante verbindet zwei Scheitelpunkte, um zu zeigen, dass zwischen ihnen eine Beziehung besteht. Kanten können in eine Richtung oder in zwei Richtungen verlaufen. Wenn die Kanten in einem Graphen alle einseitig sind, sagen wir, dass der Graph ein **gerichteter Graph** oder ein **Digraph** ist. Der oben gezeigte Graph über die Klassenvoraussetzungen ist eindeutig ein Digraph, da Sie einige Klassen vor anderen belegen müssen.

**Gewichtung**
Kanten können gewichtet werden, um zu zeigen, dass es Kosten verursacht, von einem Scheitelpunkt zum anderen zu gehen. In einem Graphen von Straßen, die eine Stadt mit einer anderen verbinden, könnte das Gewicht auf der Kante beispielsweise die Entfernung zwischen den beiden Städten darstellen.

Mit diesen Definitionen in der Hand können wir einen Graphen formal definieren. Ein Graph kann durch G dargestellt werden, wobei $G=(V,E)$. Für den Graphen $G$ ist $V$ eine Menge von Eckpunkten und $E$ eine Menge von Kanten. Jede Kante ist ein Tupel $(v,w)$, wobei $w,v ∈V$. Wir können dem Kanten-Tupel eine dritte Komponente hinzufügen, um ein Gewicht darzustellen. Ein Untergraf $s$ ist eine Menge von Kanten $e$ und Scheitelpunkten $v$, so dass $e⊂E$ und $v⊂V$.

*Abbildung 2* zeigt ein weiteres Beispiel für einen einfachen gewichteten Digraphen. Formal können wir diesen Graphen als die Menge von sechs Eckpunkten darstellen: $$V={V0,V1,V2,V3,V4,V5}$$
und den Satz von neun Kanten:
$$E=\{(v0,v1,5),(v1,v2,4),(v2,v3,9),(v3,v4,7),(v4,v0,1),(v0,v5,2),(v5,v4,8),(v3,v5,3),(v5,v2,1)\}$$

<center><img src="Bilder/Graphen/graphen02.PNG"></center>
<center>Abbildung 2: Ein einfaches Beispiel für einen gerichteten Graph</center>

Das Beispielgraph in *Abbildung 2* dient zur Veranschaulichung zweier weiterer wichtiger Graphenbegriffe:

**Pfad**
Ein Pfad in einem Graphen ist eine Folge von Eckpunkten, die durch Kanten verbunden sind. Formal würden wir einen Pfad als $w1,w2,....,wn$ definieren, so dass $(wi,wi+1)∈E$ für alle $1≤i≤n-1$. Die ungewichtete Pfadlänge ist die Anzahl der Kanten im Pfad, konkret $n-1$. Die gewichtete Pfadlänge ist die Summe der Gewichte aller Kanten im Pfad. Zum Beispiel ist in *Abbildung 2* der Pfad von $V3$ bis $V1$ die Abfolge der Eckpunkte $(V3,V4,V0,V1)$. Die Kanten sind $\{(v3,v4,7),(v4,v0,1),(v0,v1,5)\}$.

**Zyklus**
Ein Zyklus in einem gerichteten Graphen ist ein Pfad, der am gleichen Scheitelpunkt beginnt und endet. Zum Beispiel ist in *Abbildung 2* der Pfad $(V5,V2,V3,V5$ ein Zyklus. Ein Graph ohne Zyklen wird als **azyklischer Graph** bezeichnet. Ein gerichteter Graph ohne Zyklen wird als **gerichteter azyklischer Graph** oder **DAG** bezeichnet. Wir werden sehen, dass wir mehrere wichtige Probleme lösen können, wenn das Problem als DAG dargestellt werden kann.


<a id='abstrakteDatentyp'></a>
## Der abstrakte Datentyp Graph

Der abstrakte Datentyp des Graphen (ADT) ist wie folgt definiert:
* `Graph()` erzeugt einen neuen, leeren Graphen.
* `addVertex(vert)` fügt dem Graphen eine Instanz von Vertex hinzu.
* `addEdge(fromVert, toVert)` fügt dem Graphen eine neue, gerichtete Kante hinzu, die zwei Vertices verbindet.
* `addEdge(fromVert, toVert, weight)` fügt dem Graphen, der zwei Eckpunkte verbindet, eine neue, gewichtete, gerichtete Kante hinzu.
* `getVertex(vertKey)` findet den Knoten im Graphen mit dem Namen `vertKey`.
* `getVertices()` gibt die Liste aller Eckpunkte im Graphen zurück.
* `in` gibt `True` für eine Aussage des Formexpunktes im Graphen zurück, wenn der gegebene Punkt im Graphen ist, andernfalls `False`.

Beginnend mit der formalen Definition für einen Graphen gibt es mehrere Möglichkeiten, wie wir den Graphen ADT in Python implementieren können. Wir werden sehen, dass es Kompromisse bei der Verwendung unterschiedlicher Darstellungen zur Implementierung des oben beschriebenen ADT gibt. Es gibt zwei bekannte Implementierungen eines Graphen, die Adjazenzmatrix und die Adjazenzliste. Wir werden beide Optionen erklären und dann eine davon als Python-Klasse implementieren

<a id='Adjazenzmatrix'></a>
## Eine Adjazenzmatrix

Eine der einfachsten Möglichkeiten, einen Graphen zu implementieren, ist die Verwendung einer zweidimensionalen Matrix. Bei dieser Matrix-Implementierung stellt jede der Zeilen und Spalten einen Scheitelpunkt im Graphen dar. Der Wert, der in der Zelle am Schnittpunkt von Zeile $v$ und Spalte $w$ gespeichert ist, zeigt an, ob es eine Kante von Scheitelpunkt $v$ zu Scheitelpunkt $w$ gibt. Wenn zwei Scheitelpunkte durch eine Kante verbunden sind, sagen wir, dass sie **adjazent (benachbart)** sind. *Abbildung 3* veranschaulicht die Adjazenzmatrix für den Graphen in *Abbildung 2*. Ein Wert in einer Zelle stellt das Gewicht der Kante von Scheitelpunkt $v$ bis Scheitelpunkt $w$ dar.

<center><img src="Bilder/Graphen/graphen03.PNG"></center>
<center>Abbildung 3: Eine Adjazenzmatrix-Darstellung für einen Graphen</center>

Der Vorteil der Adjazenzmatrix ist, dass sie einfach ist und bei kleinen Graphen leicht zu erkennen ist, welche Knoten mit anderen Knoten verbunden sind. Beachten Sie jedoch, dass die meisten Zellen in der Matrix leer sind. Da die meisten Zellen leer sind, sagen wir, dass diese Matrix "spärlich" ist. Eine Matrix ist keine sehr effiziente Methode, um spärliche Daten zu speichern. Tatsächlich müssen Sie in Python alles daran setzen, um überhaupt eine Matrixstruktur wie die in Abbildung 3 zu erstellen.

Die Adjazenzmatrix ist eine gute Implementierung für einen Graphen, wenn die Anzahl der Kanten groß ist. Aber was verstehen wir unter groß? Wie viele Kanten wären nötig, um die Matrix zu füllen? Da es für jeden Scheitelpunkt im Graphen eine Zeile und eine Spalte gibt, ist die Anzahl der zum Füllen der Matrix erforderlichen Kanten $|V|^2$. Eine Matrix ist voll, wenn jeder Eckpunkt mit jedem anderen Eckpunkt verbunden ist. Es gibt nur wenige echte Probleme, die sich dieser Art von Konnektivität nähern. Die Probleme, die wir in diesem Kapitel betrachten werden, betreffen alle Graphen, die nur spärlich miteinander verbunden sind.

<a id='Adjazenzliste'></a>
## Eine Adjazenzliste

Eine platzsparendere Art, einen spärlich verbundenen Graphen zu implementieren, ist die Verwendung einer Adjazenzliste. In einer Adjazenzlisten-Implementierung führen wir eine Hauptliste aller Scheitelpunkte im Graph-Objekt, und dann führt jedes Scheitelpunkt-Objekt im Graph eine Liste der anderen Scheitelpunkte, mit denen es verbunden ist. In unserer Implementierung der `Vertex`-Klasse verwenden wir ein Wörterbuch statt einer Liste, wobei die Schlüssel des Wörterbuchs die Vertices und die Werte die Gewichte sind. *Abbildung 4* veranschaulicht die Darstellung der Adjazenzliste für den Graphen in *Abbildung 2*.

<center><img src="Bilder/Graphen/graphen04.PNG"></center>
<center>Abbildung 4: Eine Adjazenzlistendarstellung eines Graphen</center>

Der Vorteil der Implementierung der Adjazenzliste besteht darin, dass wir damit einen spärlichen Graphen kompakt darstellen können. Die Adjazenzliste ermöglicht es uns auch, alle Links, die direkt mit einem bestimmten Vertex verbunden sind, leicht zu finden.

<a id='Implementierung'></a>
## Implementierung

Mit Hilfe von Wörterbüchern ist es einfach, die Adjazenzliste in Python zu implementieren. In unserer Implementierung des abstrakten Datentyps Graph werden wir zwei Klassen erstellen (siehe *Auflistung 1* und *Auflistung 2*), Graph, der die Hauptliste der Vertices enthält, und Vertex, der jeden Vertex im Graph darstellt.

Jeder `Vertex` verwendet ein Wörterbuch, um die Eckpunkte, mit denen er verbunden ist, und das Gewicht jeder Kante zu verfolgen. Dieses Wörterbuch wird als `connectedTo` bezeichnet. Die folgende Auflistung zeigt den Code für die `Vertex`-Klasse. Der Konstruktor initialisiert einfach die `Id`, die normalerweise eine Zeichenfolge ist, und das Wörterbuch `connectedTo`. Die `addNeighbor`-Methode wird verwendet, um eine Verbindung von diesem Vertex zu einem anderen hinzuzufügen. Die Methode `getConnections` gibt alle Eckpunkte in der Adjazenzliste zurück, wie sie durch die Instanzvariable `connectedTo` dargestellt werden. Die `getWeight`-Methode gibt das Gewicht der Kante von diesem Scheitelpunkt zu dem als Parameter übergebenen Eckpunkt zurück.

**Auflistung 1:**
```python
class Vertex:
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}

    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight

    def __str__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])

    def getConnections(self):
        return self.connectedTo.keys()

    def getId(self):
        return self.id

    def getWeight(self,nbr):
        return self.connectedTo[nbr]
```

Die `Graph`-Klasse, die in der nächsten Auflistung gezeigt wird, enthält ein Wörterbuch, das Vertex-Namen auf Vertex-Objekte abbildet. In *Abbildung 4* wird dieses Wörterbuchobjekt durch das schattierte graue Kästchen dargestellt. Graph bietet auch Methoden zum Hinzufügen von Vertices zu einem Graphen und zum Verbinden eines Vertex mit einem anderen. Die `getVertices`-Methode gibt die Namen aller Knoten im Graphen zurück. Darüber hinaus haben wir die `__iter__`-Methode implementiert, um die Iteration über alle Knotenobjekte in einem bestimmten Graphen zu erleichtern. Zusammen ermöglichen die beiden Methoden die Iteration über die Eckpunkte in einem Graphen anhand der Namen oder anhand der Objekte selbst.

**Auflistung 2:**
```python
class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self,key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self,n):
        return n in self.vertList

    def addEdge(self,f,t,weight=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], weight)

    def getVertices(self):
        return self.vertList.keys()

    def __iter__(self):
        return iter(self.vertList.values())
```

Unter Verwendung der soeben definierten `Graph`- und `Vertex`-Klassen erstellt die folgende Python-Session den Graphen in Abbildung 2. Zuerst erstellen wir sechs Vertices mit den Nummern 0 bis 5. Dann zeigen wir das Vertex-Wörterbuch an. Beachten Sie, dass wir für jeden Schlüssel 0 bis 5 eine Instanz eines `Vertex` erzeugt haben. Als Nächstes fügen wir die Kanten hinzu, die die Vertices miteinander verbinden. Zuletzt wird in einer verschachtelten Schleife überprüft, ob jede Kante im Graphen richtig gespeichert ist. Sie sollten die Ausgabe der Kantenliste am Ende dieser Sitzung anhand von *Abbildung 2* überprüfen.

```python
>>> g = Graph()
>>> for i in range(6):
...    g.addVertex(i)
>>> g.vertList
{0: <adjGraph.Vertex instance at 0x41e18>,
 1: <adjGraph.Vertex instance at 0x7f2b0>,
 2: <adjGraph.Vertex instance at 0x7f288>,
 3: <adjGraph.Vertex instance at 0x7f350>,
 4: <adjGraph.Vertex instance at 0x7f328>,
 5: <adjGraph.Vertex instance at 0x7f300>}
>>> g.addEdge(0,1,5)
>>> g.addEdge(0,5,2)
>>> g.addEdge(1,2,4)
>>> g.addEdge(2,3,9)
>>> g.addEdge(3,4,7)
>>> g.addEdge(3,5,3)
>>> g.addEdge(4,0,1)
>>> g.addEdge(5,4,8)
>>> g.addEdge(5,2,1)
>>> for v in g:
...    for w in v.getConnections():
...        print("( %s , %s )" % (v.getId(), w.getId()))
...
( 0 , 5 )
( 0 , 1 )
( 1 , 2 )
( 2 , 3 )
( 3 , 4 )
( 3 , 5 )
( 4 , 0 )
( 5 , 4 )
( 5 , 2 )
```

<a id='Wort-Leiter-Problematik'></a>
## Die Wort-Leiter-Problematik

Zu Beginn unserer Untersuchung von Graphenalgorithmen betrachten wir das folgende Rätsel, das als Wort-Leiter bezeichnet wird. Transformieren Sie das Wort "FOOL" in das Wort "SAGE". Bei einem Wort-Leiter-Rätsel müssen Sie die Änderung schrittweise vornehmen, indem Sie einen Buchstaben nach dem anderen ändern. Bei jedem Schritt müssen Sie ein Wort in ein anderes Wort umwandeln, es ist Ihnen nicht erlaubt, ein Wort in ein Nicht-Wort umzuwandeln. Das Wort Leiterrätsel wurde 1878 von Lewis Carroll, dem Autor von Alice im Wunderland, erfunden. Die folgende Wortfolge zeigt eine mögliche Lösung für das oben aufgeworfene Problem.
```
FOOL
POOL
POLL
POLE
PALE
SALE
SAGE
```
Es gibt viele Variationen des Wortleiterrätsels. Zum Beispiel könnte Ihnen eine bestimmte Anzahl von Schritten vorgegeben werden, in denen Sie die Umwandlung durchführen müssen, oder Sie müssen ein bestimmtes Wort verwenden. In diesem Abschnitt sind wir daran interessiert, die kleinste Anzahl von Transformationen herauszufinden, die erforderlich ist, um das Anfangswort in das Endwort zu verwandeln.

Da sich dieses Kapitel mit Graphen befasst, ist es nicht überraschend, dass wir dieses Problem mit einem Graph-Algorithmus lösen können. Hier ist ein Überblick darüber, wo wir hingehen:

* Die Beziehungen zwischen den Wörtern als Graph darstellen.
* Verwendung des Graph-Algorithmus, der als Breitensuche bekannt ist, um einen effizienten Weg vom Anfangswort zum Endwort zu finden.

<a id='ErstellenDesWort-Leiter-Graphen'></a>
## Erstellen des Wort-Leiter-Graphen

Unser erstes Problem besteht darin, herauszufinden, wie wir eine große Wortsammlung in einen Graphen verwandeln können. Wir möchten von einem Wort zum anderen eine Kante haben, wenn sich die beiden Wörter nur durch einen einzigen Buchstaben unterscheiden. Wenn wir einen solchen Graphen erstellen können, dann ist jeder Weg von einem Wort zum anderen eine Lösung für das Wort-Leiter-Rätsel. *Abbildung 5* zeigt einen kleinen Graphen mit einigen Wörtern, die das Wortleiterproblem FOOL to SAGE lösen. Beachten Sie, dass es sich bei dem Graphen um einen ungerichteten Graphen handelt und dass die Kanten nicht gewichtet sind.

<center><img src="Bilder/Graphen/graphen05.PNG"></center>
<center>Abbildung 5: Ein kleiner Wortleiter-Graph</center>

Wir könnten mehrere verschiedene Ansätze verwenden, um den Graphen zu erstellen, den wir zur Lösung dieses Problems benötigen. Lassen Sie uns mit der Annahme beginnen, dass wir eine Liste von Wörtern haben, die alle gleich lang sind. Als Ausgangspunkt können wir für jedes Wort in der Liste einen Scheitelpunkt im Graphen erstellen. Um herauszufinden, wie wir die Wörter miteinander verbinden können, könnten wir jedes Wort in der Liste mit jedem anderen vergleichen. Wenn wir vergleichen, wollen wir sehen, wie viele Buchstaben unterschiedlich sind. Wenn sich die beiden fraglichen Wörter nur um einen Buchstaben unterscheiden, können wir im Graphen eine Kante zwischen ihnen erzeugen. Für eine kleine Gruppe von Wörtern würde dieser Ansatz gut funktionieren; nehmen wir jedoch an, wir hätten eine Liste von 5.110 Wörtern. Grob gesagt ist der Vergleich eines Wortes mit jedem anderen Wort auf der Liste ein $O(n^2)$-Algorithmus. Bei 5.110 Wörtern ergibt $n^2$ mehr als 26 Millionen Vergleiche.

Wir können es viel besser machen, wenn wir den folgenden Ansatz verwenden. Nehmen wir an, wir haben eine riesige Anzahl von Eimern, von denen jeder ein Wort mit vier Buchstaben auf der Außenseite hat, außer dass einer der Buchstaben auf dem Etikett durch einen Unterstrich ersetzt wurde. Betrachten Sie zum Beispiel *Abbildung 6*, wir könnten einen Eimer mit der Bezeichnung "pop_" haben. Während wir jedes Wort in unserer Liste verarbeiten, vergleichen wir das Wort mit jedem Eimer, wobei wir das "\_" als Platzhalter verwenden, so dass sowohl "pope" als auch "pops" mit "pop_" übereinstimmen würden. Jedes Mal, wenn wir einen passenden Eimer finden, legen wir unser Wort in diesen Eimer. Sobald wir alle Wörter in den entsprechenden Eimern haben, wissen wir, dass alle Wörter in dem Eimer miteinander verbunden sein müssen.

<center><img src="Bilder/Graphen/graphen06.PNG"></center>
<center>Abbildung 6: Worteimer für Wörter, die sich durch einen Buchstaben unterscheiden</center>

In Python können wir das gerade beschriebene Schema mit Hilfe eines Wörterbuchs implementieren. Die Etiketten auf den Eimern, die wir gerade beschrieben haben, sind die Schlüssel in unserem Wörterbuch. Der für diesen Schlüssel gespeicherte Wert ist eine Liste von Wörtern. Sobald wir das Wörterbuch aufgebaut haben, können wir den Graph erstellen. Wir beginnen unseren Graphen, indem wir einen Eckpunkt für jedes Wort im Graphen erstellen. Dann erstellen wir Kanten zwischen allen Eckpunkten, die wir für Wörter finden, die unter demselben Schlüssel im Wörterbuch stehen. *Auflistung 3* zeigt den zur Erstellung des Graphen erforderlichen Python-Code.

**Auflistung 3**
```python
from pythonds.graphs import Graph

def buildGraph(wordFile):
    d = {}
    g = Graph()
    wfile = open(wordFile,'r')
    # create buckets of words that differ by one letter
    for line in wfile:
        word = line[:-1]
        for i in range(len(word)):
            bucket = word[:i] + '_' + word[i+1:]
            if bucket in d:
                d[bucket].append(word)
            else:
                d[bucket] = [word]
    # add vertices and edges for words in the same bucket
    for bucket in d.keys():
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1 != word2:
                    g.addEdge(word1,word2)
    return g
```

Da dies unser erstes Problem mit Graphen aus der realen Welt ist, fragen Sie sich vielleicht, wie spärlich der Graph ist? Die Liste der Wörter mit vier Buchstaben, die wir für dieses Problem haben, ist 5.110 Wörter lang. Wenn wir eine Adjazenzmatrix verwenden würden, hätte die Matrix 5.110 * 5.110 = 26.112.100 Zellen. Der durch die Funktion `buildGraph` konstruierte Graph hat genau 53.286 Kanten, so dass die Matrix nur 0,20 % der Zellen gefüllt wäre! Das ist in der Tat eine sehr spärliche Matrix.

<a id='ImplementierungBreadthFirstSearch'></a>
## Implementierung von Breadth First Search

Mit dem erstellten Graphen können wir uns nun dem Algorithmus zuwenden, den wir verwenden werden, um die kürzeste Lösung für das Wort-Leiter-Problem zu finden. Der Algorithmus für den Graphen, den wir verwenden werden, heißt " breadth first search"-Algorithmus. Die **Breadth first search** (BFS) ist einer der einfachsten Algorithmen für die Suche in einem Graphen. Er dient auch als Prototyp für mehrere andere wichtige Graph-Algorithmen, die wir später untersuchen werden.

Bei einem Graphen $G$ und einem Startpunkt $s$ werden bei einer Breitensuche die Kanten im Graphen untersucht, um alle Punkte in $G$ zu finden, für die es einen Pfad von $s$ aus gibt. Das Bemerkenswerte an einer Breitensuche ist, dass sie alle Punkte findet, die eine Entfernung $k$ von $s$ haben, bevor sie alle Punkte findet, die eine Entfernung $k+1$ haben. Eine gute Möglichkeit, sich vorzustellen, was der Algorithmus der ersten Breitensuche tut, ist, sich vorzustellen, dass er einen Baum aufbaut, eine Ebene des Baumes nach der anderen. Bei der ersten Breitensuche werden alle Kinder des Startpunktes hinzugefügt, bevor der Algorithmus beginnt, eines der Enkelkinder zu entdecken.

Um seinen Fortschritt zu verfolgen, färbt BFS jeden der Eckpunkte weiß, grau oder schwarz ein. Alle Eckpunkte werden bei ihrer Konstruktion weiß initialisiert. Ein weißer Scheitelpunkt ist ein unentdeckter Scheitelpunkt. Wenn ein Scheitelpunkt anfänglich entdeckt wird, wird er grau gefärbt, und wenn BFS einen Scheitelpunkt vollständig erforscht hat, wird er schwarz gefärbt. Das bedeutet, dass ein schwarz gefärbter Scheitelpunkt keine weißen Scheitelpunkte mehr neben ihm hat. Ein grauer Knoten hingegen kann einige weiße Scheitelpunkte neben sich haben, was darauf hinweist, dass es noch weitere Scheitelpunkte zu erforschen gibt.

Der in *Auflistung 4* unten gezeigte Algorithmus für die "breadth first search" verwendet die von uns zuvor entwickelte Darstellung des Graphen der Adjazenzliste. Zusätzlich verwendet er eine `Queue`, ein entscheidender Punkt, wie wir sehen werden, um zu entscheiden, welcher Vertex als nächstes untersucht werden soll.

Darüber hinaus verwendet der BFS-Algorithmus eine erweiterte Version der `Vertex`-Klasse. Diese neue Vertex-Klasse fügt drei neue Instanzvariablen hinzu: Abstand, Vorgänger und Farbe. Jede dieser Instanzvariablen verfügt auch über die entsprechenden Getter- und Setter-Methoden. Der Code für diese erweiterte Vertex-Klasse ist im `pythonds`-Paket enthalten, aber wir werden ihn Ihnen hier nicht zeigen, da es nichts Neues zu lernen gibt, wenn Sie sich die zusätzlichen Instanzvariablen ansehen.

Die BFS beginnt am Startpunkt `s` und `start` wird grau eingefärbt, um zu zeigen, dass er derzeit erforscht wird. Zwei andere Werte, der Abstand und der Vorgänger, werden für den Start-Eckpunkt auf 0 bzw. `None` initialisiert. Schließlich wird der Start in eine `Queue` gestellt. Der nächste Schritt besteht darin, mit der systematischen Erkundung der Vertices am Anfang der Warteschlange zu beginnen. Wir erforschen jeden neuen Knoten am Anfang der Warteschlange, indem wir über seine Adjazenzliste iterieren. Während jeder Knoten auf der Adjazenzliste untersucht wird, wird seine Farbe überprüft. Wenn er weiß ist, ist der Knoten unerforscht, und es geschehen vier Dinge:

1. Der neue, unerforschte Scheitelpunkt `nbr`, wird grau gefärbt.
2. Der Vorgänger von `nbr` wird auf den aktuellen Knoten `currentVert` gesetzt.
3. Der Abstand zu `nb`r wird auf den Abstand zu `currentVert + 1` gesetzt
4. `nb`r wird am Ende einer Warteschlange hinzugefügt. Durch das Hinzufügen von `nbr` am Ende der Warteschlange wird dieser Knoten effektiv für die weitere Erkundung terminiert, jedoch erst dann, wenn alle anderen Knoten auf der Adjazenzliste von `currentVert` erkundet worden sind.

**Auflistung 4**
```python
from pythonds.basic import Queue

def bfs(g,start):
    start.setDistance(0)
    start.setPred(None)
    vertQueue = Queue()
    vertQueue.enqueue(start)
    while (vertQueue.size() > 0):
        currentVert = vertQueue.dequeue()
        for nbr in currentVert.getConnections():
            if (nbr.getColor() == 'white'):
                nbr.setColor('gray')
                nbr.setDistance(currentVert.getDistance() + 1)
                nbr.setPred(currentVert)
                vertQueue.enqueue(nbr)
        currentVert.setColor('black')
```

Schauen wir uns an, wie die `bfs`-Funktion den ersten Breitenbaum konstruieren würde, der dem Graphen in *Abbildung 5* entspricht. Ausgehend von fool nehmen wir alle Knoten, die an fool angrenzen, und fügen sie dem Baum hinzu. Zu den angrenzenden Knoten gehören Pool, Folie, Foul und Cool. Jeder dieser Knoten wird in die Warteschlange der neuen zu erweiternden Knoten aufgenommen. *Abbildung 7* zeigt den Zustand des in Bearbeitung befindlichen Baums zusammen mit der Warteschlange nach diesem Schritt.

<center><img src="Bilder/Graphen/graphen07.PNG"></center>
<center>Abbildung 7: Der erste Schritt der Breadth First Suche</center>

Im nächsten Schritt entfernt bfs den nächsten Knoten (Pool) aus dem vorderen Teil der Warteschlange und wiederholt den Vorgang für alle seine benachbarten Knoten. Wenn bfs jedoch den Knoten cool untersucht, stellt es fest, dass die Farbe von cool bereits auf grau geändert wurde. Dies deutet darauf hin, dass es einen kürzeren Weg zu Cool gibt und dass Cool sich bereits in der Warteschlange für eine weitere Expansion befindet. Der einzige neue Knoten, der während der Untersuchung des Pools zur Warteschlange hinzugefügt wurde, ist Poll. Der neue Zustand des Baums und der Warteschlange ist in *Abbildung 8* dargestellt.

<center><img src="Bilder/Graphen/graphen08.PNG"></center>
<center>Abbildung 8: Der zweite Schritt der Breadth First Suche</center>

Der nächste Scheitelpunkt in der Warteschlange ist die foil. Der einzige neue Knoten, den Foil dem Baum hinzufügen kann, ist fail. Wenn `bfs` mit der Verarbeitung der Warteschlange fortfährt, fügt keiner der beiden nächsten Knoten der Warteschlange oder dem Baum etwas Neues hinzu. *Abbildung 9* zeigt den Baum und die Warteschlange nach dem Expandieren aller Knoten auf der zweiten Ebene des Baums.

<center><img src="Bilder/Graphen/graphen09.PNG"></center>
<center>Abbildung 9: Der Breadth First Suchbaum nach Abschluss einer Stufe</center>

<center><img src="Bilder/Graphen/graphen10.PNG"></center>
<center>Abbildung 10: Der finale Breadth First Suchbaum</center>

Sie sollten den Algorithmus selbstständig weiter durcharbeiten, damit Sie mit seiner Funktionsweise vertraut sind. *Abbildung 10* finalen Breadth First Suchbaum, nachdem alle Vertices in Abbildung 3 erweitert wurden. Das Erstaunliche an der Breadth First Suche Lösung ist, dass wir nicht nur das FOOL-SAGE-Problem, mit dem wir begonnen haben, gelöst haben, sondern dass wir im Laufe der Zeit auch viele andere Probleme gelöst haben. Wir können an jedem beliebigen Scheitelpunkt im Breiten-Erst-Suchbaum beginnen und den Vorgängerpfeilen zurück zur Wurzel folgen, um die kürzeste Wort-Leiter von jedem Wort zurück zu FOOL zu finden. Die Funktion unten (*Auflistung 5*) zeigt, wie man den Vorgänger-Links folgt, um die Wort-Leiter auszudrucken.

**Auflistung 5**
```python
def traverse(y):
    x = y
    while (x.getPred()):
        print(x.getId())
        x = x.getPred()
    print(x.getId())

traverse(g.getVertex('sage'))
```

<a id='AnalyseBreadthFirstSearch'></a>
## Analyse der Breadth First Suche

Bevor wir mit anderen Graph-Algorithmen fortfahren, sollten wir die Laufzeitleistung des breadth first-Suchalgorithmus analysieren. Als erstes ist zu beobachten, dass die while-Schleife höchstens einmal für jeden Knoten im Graphen $|V|$ ausgeführt wird. Sie können sehen, dass dies zutrifft, da ein Knoten weiß sein muss, bevor er untersucht und der Warteschlange hinzugefügt werden kann. Dies ergibt $O(V)$ für die while-Schleife. Die for-Schleife, die innerhalb der while-Schleife verschachtelt ist, wird höchstens einmal für jede Kante im Graphen ausgeführt, $|E|$. Der Grund dafür ist, dass jeder Knoten höchstens einmal aus der Warteschlange entfernt wird und wir eine Kante vom Knoten $u$ zum Knoten $v$ nur untersuchen, wenn der Knoten $u$ aus der Warteschlange entfernt wird. Dies ergibt $O(E)$ für die for-Schleife. Die Kombination der beiden Schleifen ergibt $O(V+E)$.

Natürlich ist die Breadth First Suche nur ein Teil der Aufgabe. Den Verbindungen vom Startknoten zum Zielknoten zu folgen, ist der andere Teil der Aufgabe. Der schlimmste Fall dafür wäre, wenn der Graph eine einzige lange Kette wäre. In diesem Fall wäre das Durchqueren aller Eckpunkte $O(V)$. Der Normalfall wird ein Bruchteil von $|V|$ sein, aber wir würden trotzdem $O(V)$ schreiben.

Schließlich gibt es, zumindest für dieses Problem, die Zeit, die benötigt wird, um den anfänglichen Graphen zu erstellen. Wir überlassen die Analyse der Funktion `buildGraph` als Übung für Sie.

<a id='Rittertour-Problem'></a>
## Das Rittertour-Problem

Ein weiteres klassisches Problem, das wir zur Veranschaulichung eines zweiten gängigen Graphen-Algorithmus verwenden können, ist die "Rittertour". Das Rittertour-Puzzle wird auf einem Schachbrett mit einer einzigen Schachfigur, dem Springer, gespielt. Das Ziel des Puzzles besteht darin, eine Zugfolge zu finden, die es dem Springer ermöglicht, jedes Feld auf dem Brett genau einmal zu besuchen. Eine solche Sequenz wird als "Tour" bezeichnet. Das Rätsel der Springertour fasziniert seit vielen Jahren Schachspieler, Mathematiker und Informatiker gleichermaßen. Die Obergrenze für die Anzahl der möglichen legalen Touren für ein acht-auf-Acht-Schachbrett beträgt bekanntlich $1,305×10^35$; es gibt jedoch noch mehr mögliche Sackgassen. Offensichtlich ist dies ein Problem, das einige echte Köpfe, einige echte Rechenleistung oder beides erfordert.

Obwohl Forscher viele verschiedene Algorithmen zur Lösung des Rittertour-Problems untersucht haben, ist eine Graphensuche eine der am einfachsten zu verstehenden und zu programmierenden. Auch hier werden wir das Problem in zwei Hauptschritten lösen:

* Darstellung der legalen Züge eines Springers auf einem Schachbrett als Graph.
* Verwenden eines Graph-Algorithmus, um einen Pfad der Länge Zeilen×Spalten-1 zu finden, bei dem jeder Scheitelpunkt auf dem Graphen genau einmal besucht wird.

<a id='GraphenDerRittertour'></a>
## Den Graphen der Rittertour erstellen

Um das Rittertour-Problem als Graph darzustellen, werden wir die folgenden beiden Ideen verwenden: Jedes Feld auf dem Schachbrett kann als ein Knoten im Graphen dargestellt werden. Jeder legale Zug des Springers kann als eine Kante im Graphen dargestellt werden. *Abbildung 11* veranschaulicht die legalen Züge eines Springers und die entsprechenden Kanten in einem Graphen.

<center><img src="Bilder/Graphen/graphen11.PNG"></center>
<center>Abbildung 11: Legale Züge für einen Ritter auf Feld 12 und der entsprechende Graph</center>

Um den vollständigen Graph für ein n-by-n-Board zu erstellen, können wir die in *Auflistung 6* gezeigte Python-Funktion verwenden. Die Funktion knightGraph führt einen Durchgang über das gesamte Brett aus. Auf jedem Feld auf dem Brett ruft die Funktion knightGraph einen Helfer, genLegalMoves, auf, um eine Liste der legalen Züge für diese Stellung auf dem Brett zu erstellen. Alle legalen Züge werden dann im Graphen in Kanten umgewandelt. Eine weitere Hilfsfunktion posToNodeId wandelt eine Stellung auf dem Brett in Form einer Zeile und einer Spalte in eine lineare Eckpunktnummer um, ähnlich den in *Abbildung 11* gezeigten Eckpunktnummern.

**Auflistung 6**
```python
from pythonds.graphs import Graph

def knightGraph(bdSize):
    ktGraph = Graph()
    for row in range(bdSize):
           for col in range(bdSize):
                nodeId = posToNodeId(row,col,bdSize)
                newPositions = genLegalMoves(row,col,bdSize)
                for e in newPositions:
                    nid = posToNodeId(e[0],e[1],bdSize)
                    ktGraph.addEdge(nodeId,nid)
    return ktGraph

def posToNodeId(row, column, board_size):
    return (row * board_size) + column
```

Die Funktion `genLegalMoves` (*Auflistung 7*) nimmt die Stellung des Springers auf dem Brett ein und generiert jeden der acht möglichen Züge. Die Hilfsfunktion `legalCoord` (*Auflistung 7*) stellt sicher, dass ein bestimmter Zug, der generiert wird, noch auf dem Brett ist.

**Auflistung 7**
```python
def genLegalMoves(x,y,bdSize):
    newMoves = []
    moveOffsets = [(-1,-2),(-1,2),(-2,-1),(-2,1),
                   ( 1,-2),( 1,2),( 2,-1),( 2,1)]
    for i in moveOffsets:
        newX = x + i[0]
        newY = y + i[1]
        if legalCoord(newX,bdSize) and \
                        legalCoord(newY,bdSize):
            newMoves.append((newX,newY))
    return newMoves

def legalCoord(x,bdSize):
    if x >= 0 and x < bdSize:
        return True
    else:
        return False
```

*Abbildung 12* zeigt den vollständigen Graphen der möglichen Züge auf einem acht-x-acht Brett. Es gibt genau 336 Kanten im Graphen. Beachten Sie, dass die Eckpunkte, die den Kanten des Bretts entsprechen, weniger Verbindungen (legale Züge) haben als die Eckpunkte in der Mitte des Bretts. Man sieht wieder einmal, wie spärlich der Graph ist. Wenn der Graph vollständig verbunden wäre, gäbe es 4.096 Kanten. Da es nur 336 Kanten gibt, wäre die Adjazenzmatrix nur zu 8,2 Prozent gefüllt.

<center><img src="Bilder/Graphen/graphen12.PNG"></center>
<center>Abbildung 12: Alle legalen Züge für einen Springer auf einem 8×8-Schachbrett</center>

<a id='ImplementierungDerRittertour'></a>
## Implementierung der Rittertour

Der Suchalgorithmus, den wir zur Lösung des Rittertour-Problems verwenden werden, heißt **Depth First Suche** (**DFS**). Während der im vorigen Abschnitt besprochene Suchalgorithmus "Breadth First" einen Suchbaum Ebene für Ebene aufbaut, wird bei der "Tiefe zuerst" ein Suchbaum erstellt, indem ein Ast des Baums so tief wie möglich erforscht wird. In diesem Abschnitt werden wir uns zwei Algorithmen ansehen, die eine Suche in der Tiefe zuerst implementieren. Der erste Algorithmus, den wir uns direkt ansehen werden, löst das Rittertour-Problem, indem er ausdrücklich verbietet, einen Knoten mehr als einmal zu besuchen. Die zweite Implementierung ist allgemeiner, erlaubt jedoch, dass Knoten während des Aufbaus des Baums mehr als einmal besucht werden können. Die zweite Version wird in den folgenden Abschnitten verwendet, um zusätzliche Graph-Algorithmen zu entwickeln.

Die erste Tiefenexploration des Graphen ist genau das, was wir brauchen, um einen Pfad zu finden, der genau 63 Kanten hat. Wir werden sehen, dass der Suchalgorithmus der ersten Tiefensuche, wenn er eine Sackgasse findet (eine Stelle im Graphen, an der keine Bewegungen mehr möglich sind), den Baum zum nächsttieferen Scheitelpunkt zurückführt, der es ihm erlaubt, eine legale Bewegung zu machen.

Die Funktion `knightTour` benötigt vier Parameter: `n`, die aktuelle Tiefe im Suchbaum; `path`, eine Liste der bis zu diesem Punkt besuchten Knoten; `u`, der Knoten im Graphen, den wir erforschen möchten; und die Begrenzung der Anzahl (`limit`) der Knoten im Pfad. Die Funktion `knightTour` ist rekursiv. Wenn die Funktion `knightTour` aufgerufen wird, prüft sie zunächst die Basisfallbedingung. Wenn wir einen Pfad haben, der 64 Knoten enthält, kehren wir von `knightTour` mit dem Status `True` zurück und zeigen damit an, dass wir eine erfolgreiche Tour gefunden haben. Wenn der Pfad nicht lang genug ist, fahren wir fort, eine Ebene tiefer zu erforschen, indem wir einen neuen zu erforschenden Knoten auswählen und `knightTour` rekursiv für diesen Knoten aufrufen.

Die DFS verwendet auch Farben, um zu verfolgen, welche Knoten im Graphen besucht wurden. Nicht besuchte Vertices sind weiß und besuchte Vertices sind grau gefärbt. Wenn alle Nachbarn eines bestimmten Vertex untersucht wurden und wir unsere Ziellänge von 64 Vertices noch nicht erreicht haben, sind wir in einer Sackgasse angelangt. Wenn wir eine Sackgasse erreichen, müssen wir zurückgehen. Rückverfolgung geschieht, wenn wir mit dem Status Falsch von `knightTour` zurückkehren. In der ersten BFS haben wir eine Warteschlange benutzt, um zu verfolgen, welchen Knoten wir als nächstes besuchen müssen. Da die Depth First Suche rekursiv ist, verwenden wir implizit einen Stapel, der uns bei der Rückverfolgung hilft. Wenn wir von einem Aufruf von `knightTour` mit dem Status `False` in Zeile 11 zurückkehren, bleiben wir innerhalb der `while`-Schleife und schauen uns den nächsten Vertex in `nbrList` an.

**Auflistung 8**
```python
from pythonds.graphs import Graph, Vertex
def knightTour(n,path,u,limit):
        u.setColor('gray')
        path.append(u)
        if n < limit:
            nbrList = list(u.getConnections())
            i = 0
            done = False
            while i < len(nbrList) and not done:
                if nbrList[i].getColor() == 'white':
                    done = knightTour(n+1, path, nbrList[i], limit)
                i = i + 1
            if not done:  # prepare to backtrack
                path.pop()
                u.setColor('white')
        else:
            done = True
        return done
```

Sehen wir uns ein einfaches Beispiel von `KnightTour` in Aktion an. Sie können sich auf die untenstehenden Abbildungen beziehen, um den Schritten der Suche zu folgen. Für dieses Beispiel nehmen wir an, dass der Aufruf der Methode `getConnections` auf Zeile 6 die Knoten in alphabetischer Reihenfolge anordnet. Wir beginnen mit dem Aufruf `knightTour(0,Pfad,A,6)`.

`knightTour` beginnt mit Knoten A *Abbildung 13*. Die an A angrenzenden Knoten sind B und D. Da B alphabetisch vor D liegt, wählt DFS B aus, um als nächstes zu expandieren, wie in *Abbildung 14* dargestellt. Die Erkundung von B geschieht, wenn `knightTour` rekursiv aufgerufen wird. Da B neben C und D liegt, wählt knightTour als nächstes C zu erforschen. Wie Sie jedoch in *Abbildung 15* sehen können, ist Knoten C eine Sackgasse ohne angrenzende weiße Knoten. An diesem Punkt ändern wir die Farbe von Knoten C wieder auf weiß. Der Aufruf von `knightTour` gibt den Wert False zurück. Die Rückkehr aus dem rekursiven Aufruf verfolgt die Suche effektiv bis zum Knoten B zurück (siehe *Abbildung 16*). Der nächste zu untersuchende Knoten auf der Liste ist der Knoten D, so dass `knightTour` einen rekursiven Aufruf durchfährt und sich zum Knoten D bewegt (siehe *Abbildung 17*). Von Knoten D an kann `knightTour` weiterhin rekursive Aufrufe machen, bis wir wieder zu Knoten C gelangen (siehe *Abbildung 18*, *Abbildung 19* und *Abbildung 20*). Dieses Mal jedoch, wenn wir zu Knoten C gelangen, schlägt der Test `n < limit` fehl, so dass wir wissen, dass wir alle Knoten im Graphen erschöpft haben. An diesem Punkt können wir zu `True` zurückkehren, um anzuzeigen, dass wir einen erfolgreichen Rundgang durch den Graphen gemacht haben. Wenn wir die Liste zurückgeben, hat der `path` die Werte `[A,B,D,E,F,C]`, d.h. die Reihenfolge, in der wir den Graphen durchlaufen müssen, um jeden Knoten genau einmal zu besuchen.

<center><img src="Bilder/Graphen/graphen13.PNG"></center>
<center>Abbildung 13: Anfangen mit Knoten A</center>

<center><img src="Bilder/Graphen/graphen14.PNG"></center>
<center>Abbildung 14: B erforschen</center>

<center><img src="Bilder/Graphen/graphen15.PNG"></center>
<center>Abbildung 15: Knoten C ist eine Sackgasse</center>

<center><img src="Bilder/Graphen/graphen16.PNG"></center>
<center>Abbildung 16: Zurück zu B</center>

<center><img src="Bilder/Graphen/graphen17.PNG"></center>
<center>Abbildung 17: D erforschen</center>

<center><img src="Bilder/Graphen/graphen18.PNG"></center>
<center>Abbildung 18: E erforschen</center>

<center><img src="Bilder/Graphen/graphen19.PNG"></center>
<center>Abbildung 19: F erforschen</center>

<center><img src="Bilder/Graphen/graphen20.PNG"></center>
<center>Abbildung 20: Finish</center>

*Abbildung 21* zeigt Ihnen, wie ein kompletter Rundgang um ein achtmal acht Quadratmeter großes Brett aussieht. Es gibt viele mögliche Touren; einige sind symmetrisch. Mit einigen Modifikationen können Sie Rundgänge machen, die am gleichen Platz beginnen und enden.

<center><img src="Bilder/Graphen/graphen21.PNG"></center>
<center>Abbildung 21: Ein kompletter Durchgang durch das Board</center>

<a id='AnalyseDerRittertour'></a>
## Analyse der Rittertour

Es gibt noch ein letztes interessantes Thema zum Problem der Rittertour, dann gehen wir zur allgemeinen Version der Tiefensuche über. Das Thema ist die Performance. Insbesondere ist `knightTour` sehr empfindlich für die Methode, mit der Sie den nächsten zu besuchenden Knotenpunkt auswählen. Zum Beispiel können Sie auf einem fünf mal fünf Brett einen Pfad in etwa 1,5 Sekunden auf einem relativ schnellen Computer erstellen. Aber was passiert, wenn Sie es mit einem acht-x-acht Brett versuchen? In diesem Fall müssen Sie, abhängig von der Geschwindigkeit Ihres Computers, möglicherweise bis zu einer halben Stunde warten, bis Sie die Ergebnisse erhalten! Der Grund dafür ist, dass das Problem der Springertour, wie wir es bisher implementiert haben, ein exponentieller Algorithmus der Größe $O(k^N)$ ist, wobei N die Anzahl der Felder auf dem Schachbrett und k eine kleine Konstante ist. *Abbildung 22* kann uns helfen, uns zu veranschaulichen, warum dies so ist. Die Wurzel des Baumes stellt den Startpunkt der Suche dar. Von dort aus generiert und prüft der Algorithmus jeden der möglichen Züge, die der Springer ausführen kann. Wie wir bereits erwähnt haben, hängt die Anzahl der möglichen Züge von der Stellung des Springers auf dem Brett ab. In den Ecken gibt es nur zwei legale Züge, auf den an die Ecken angrenzenden Feldern drei und in der Mitte des Brettes acht. *Abbildung 23* zeigt die Anzahl der möglichen Züge für jede Stellung auf einem Brett. Auf der nächsten Ebene des Baumes gibt es wiederum zwischen 2 und 8 mögliche nächste Züge von der Position aus, die wir gerade untersuchen. Die Anzahl der möglichen zu untersuchenden Stellungen entspricht der Anzahl der Knoten im Suchbaum.

<center><img src="Bilder/Graphen/graphen22.PNG"></center>
<center>Abbildung 22: Ein Suchbaum für die Rittertour</center>

<center><img src="Bilder/Graphen/graphen23.PNG"></center>
<center>Abbildung 23: Anzahl der möglichen Züge für jedes Feld</center>

Wir haben bereits gesehen, dass die Anzahl der Knoten in einem binären Baum der Höhe N $2^{N+1}-1$ beträgt. Bei einem Baum mit Knoten, der bis zu acht statt zwei Kinder haben kann, ist die Anzahl der Knoten viel größer. Da der Verzweigungsfaktor jedes Knotens variabel ist, könnten wir die Anzahl der Knoten anhand eines durchschnittlichen Verzweigungsfaktors schätzen. Wichtig zu beachten ist, dass dieser Algorithmus exponentiell ist: $k^{N+1}-1$, wobei $k$ der durchschnittliche Verzweigungsfaktor für das Board ist. Schauen wir uns an, wie schnell dies wächst! Bei einem Brett von 5x5 ist der Baum 25 Ebenen tief, oder N = 24, wobei die erste Ebene als Ebene 0 gezählt wird. Der durchschnittliche Verzweigungsfaktor beträgt $k=3,8$. Die Anzahl der Knoten im Suchbaum beträgt also $3,8^25 -1$ oder $3,12×10^14$. Für ein 6x6-Brett, k=4,4, gibt es 1,5×1023 Knoten, und für ein reguläres 8x8-Schachbrett, $k=5,25$, gibt es $1,3×10^46$. Da es natürlich mehrere Lösungen für das Problem gibt, müssen wir nicht jeden einzelnen Knoten untersuchen, aber der Bruchteil der Knoten, den wir untersuchen müssen, ist nur ein konstanter Multiplikator, der die exponentielle Natur des Problems nicht ändert. Wir belassen es als Übung für Sie, um zu sehen, ob Sie $k$ in Abhängigkeit von der Tafelgröße ausdrücken können.

Glücklicherweise gibt es eine Möglichkeit, den Fall im Format acht mal acht so zu beschleunigen, dass er in weniger als einer Sekunde abläuft. In der folgenden Auflistung zeigen wir den Code, der die `knightTour` beschleunigt. Diese Funktion (siehe *Auflistung 9*), genannt `orderbyAvail`, wird anstelle des Aufrufs von `u.getConnections` in dem zuvor oben gezeigten Code verwendet. Die kritische Zeile in der Funktion `orderByAvail` ist Zeile 10. Diese Zeile stellt sicher, dass wir den nächsten Knoten auswählen, der die wenigsten verfügbaren Züge hat. Man könnte meinen, dies sei wirklich kontraproduktiv; warum nicht den Knoten mit den meisten verfügbaren Zügen auswählen? Sie können diesen Ansatz leicht ausprobieren, indem Sie das Programm selbst ausführen und die Zeile `resList.reverse()` direkt nach der Sortierung einfügen.

Das Problem bei der Verwendung des Knotenpunktes mit den meisten verfügbaren Zügen als nächster Knotenpunkt auf dem Pfad besteht darin, dass der Springer die mittleren Felder in der Regel schon früh in der Tour besucht. Wenn dies geschieht, kann der Springer leicht auf einer Seite des Brettes stranden, wo er die nicht besuchten Felder auf der anderen Seite des Brettes nicht erreichen kann. Andererseits führt der Besuch der Felder mit den wenigsten verfügbaren Zügen dazu, dass der Springer zuerst die Felder an den Rändern des Brettes besucht. Auf diese Weise wird sichergestellt, dass der Springer die schwer zugänglichen Ecken frühzeitig besucht und die mittleren Felder nur bei Bedarf über das Brett hüpfen kann. Die Nutzung dieser Art von Wissen zur Beschleunigung eines Algorithmus wird als Heuristik bezeichnet. Der Mensch verwendet Heuristiken täglich, um Entscheidungen zu treffen, heuristische Suchen werden häufig im Bereich der künstlichen Intelligenz eingesetzt. Diese spezielle Heuristik wird Warnsdorffs Algorithmus genannt, benannt nach H. C. Warnsdorff, der seine Idee 1823 veröffentlichte.

**Auflistung 9**
```python
def orderByAvail(n):
    resList = []
    for v in n.getConnections():
        if v.getColor() == 'white':
            c = 0
            for w in v.getConnections():
                if w.getColor() == 'white':
                    c = c + 1
            resList.append((c,v))
    resList.sort(key=lambda x: x[0])
    return [y[1] for y in resList]
```

<a id='AllgemeineDepthFirstSuche'></a>
## Allgemeine Depth First Suche

Die Rittertour ist ein Spezialfall einer Tiefensuche, bei der das Ziel darin besteht, den ersten Baum in der tiefsten Tiefe ohne Äste zu erzeugen. Die allgemeinere Depth First Suche ist eigentlich einfacher. Ihr Ziel ist es, so tief wie möglich zu suchen, so viele Knoten im Graphen wie möglich miteinander zu verbinden und gegebenenfalls zu verzweigen.

Es ist sogar möglich, dass bei einer Depth First Suche mehr als ein Baum erstellt wird. Wenn der Algorithmus für die **Depth First Suche** eine Gruppe von Bäumen erstellt, nennen wir dies einen Wald in der ersten Tiefe. Wie bei der Breadth First Suche wird auch bei der Depth First Suche auf Vorgänger-Links zurückgegriffen, um den Baum zu konstruieren. Darüber hinaus werden bei der Depth First Suche zwei zusätzliche Instanzvariablen in der `Vertex`-Klasse verwendet. Die neuen Instanzvariablen sind die Entdeckungs- und Endzeit. Die Entdeckungszeit verfolgt die Anzahl der Schritte im Algorithmus, bevor ein Vertex zum ersten Mal angetroffen wird. Die Endzeit ist die Anzahl der Schritte im Algorithmus, bevor ein Vertex schwarz eingefärbt wird. Wie wir nach der Betrachtung des Algorithmus sehen werden, bieten die Entdeckungs- und Endzeiten der Knoten einige interessante Eigenschaften, die wir in späteren Algorithmen verwenden können.

Der Code für unsere erste Tiefensuche ist in *Auflistung 10* dargestellt. Da die beiden Funktionen `dfs` und sein Helfer `dfsvisit` eine Variable verwenden, um die Zeit zwischen den Aufrufen von `dfsvisit` zu verfolgen, haben wir uns dafür entschieden, den Code als Methoden einer Klasse zu implementieren, die von der `Graph`-Klasse erbt. Diese Implementierung erweitert die `Graph`-Klasse durch Hinzufügen einer `time` Instanzvariablen und der beiden Methoden `dfs` und `dfsvisit`. Wenn Sie sich Zeile 11 ansehen, werden Sie feststellen, dass die `dfs`-Methode über alle Vertices im Graphen, der `dfsvisit` aufruft, auf den Knoten iteriert, die weiß sind. Der Grund dafür, dass wir über alle Knoten iterieren, anstatt einfach von einem gewählten Startknoten aus zu suchen, ist, dass wir sicherstellen wollen, dass alle Knoten im Graphen berücksichtigt werden und dass keine Knoten aus dem tiefen ersten Wald ausgelassen werden. Es mag ungewöhnlich aussehen, die Aussage `for aVertex in self` zu sehen, aber denken Sie daran, dass `self` in diesem Fall eine Instanz der Klasse `DFSGraph` ist, und die Iteration über alle Knoten in einer Instanz eines Graphen eine natürliche Sache ist.

**Auflistung 10**
```python
from pythonds.graphs import Graph
class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0

    def dfs(self):
        for aVertex in self:
            aVertex.setColor('white')
            aVertex.setPred(-1)
        for aVertex in self:
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)

    def dfsvisit(self,startVertex):
        startVertex.setColor('gray')
        self.time += 1
        startVertex.setDiscovery(self.time)
        for nextVertex in startVertex.getConnections():
            if nextVertex.getColor() == 'white':
                nextVertex.setPred(startVertex)
                self.dfsvisit(nextVertex)
        startVertex.setColor('black')
        self.time += 1
        startVertex.setFinish(self.time)
```

Obwohl unsere Implementierung von `bfs` nur daran interessiert war, Knoten zu berücksichtigen, für die es einen Pfad gibt, der zurück zum Anfang führt, ist es möglich, einen breiten ersten Wald zu erstellen, der den kürzesten Weg zwischen allen Knotenpaaren im Graphen darstellt. Wir belassen dies als eine Übung. In unseren nächsten beiden Algorithmen werden wir sehen, warum es wichtig ist, den Überblick über die Tiefe des ersten Waldes zu behalten.

Die `dfsvisit`-Methode beginnt mit einem einzelnen Knoten namens `startVertex` und erkundet alle benachbarten weißen Knoten so tief wie möglich. Wenn Sie sich den Code für `dfsvisit` genau ansehen und ihn mit breadth first Suche vergleichen, sollten Sie bemerken, dass der `dfsvisit`-Algorithmus fast identisch mit `bfs` ist, mit der Ausnahme, dass sich dfsvisit in der letzten Zeile der inneren for-Schleife rekursiv aufruft, um die Suche auf einer tieferen Ebene fortzusetzen, während `bfs` den Knoten in eine Warteschlange für eine spätere Erkundung einfügt. Interessant ist, dass dort, wo `bfs` eine Warteschlange verwendet, `dfsvisit` einen Stack verwendet. Sie sehen keinen Stack im Code, aber er ist implizit in dem rekursiven Aufruf von `dfsvisit` enthalten.

Die folgende Abfolge von Abbildungen veranschaulicht den Algorithmus für die erste Tiefensuche in Aktion für einen kleinen Graphen. In diesen Abbildungen zeigen die gepunkteten Linien Kanten an, die überprüft werden, aber der Knoten am anderen Ende der Kante wurde bereits dem Baum depth first hinzugefügt. Im Code wird dieser Test durchgeführt, indem geprüft wird, ob die Farbe des anderen Knotens nicht weiß ist.

Die Suche beginnt an Punkt A des Graphen (*Abbildung 24*). Da zu Beginn der Suche alle Vertices weiß sind, besucht der Algorithmus Vertex A. Der erste Schritt beim Besuch eines Vertex besteht darin, die Farbe auf grau zu setzen, was anzeigt, dass der Vertex erforscht wird und die Entdeckungszeit auf 1 gesetzt ist. Da Vertex A zwei benachbarte Vertices (B, D) hat, muss jeder dieser Vertices ebenfalls besucht werden. Wir werden die willkürliche Entscheidung treffen, dass wir die benachbarten Scheitelpunkte in alphabetischer Reihenfolge besuchen werden.

Punkt B wird als nächstes besucht (*Abbildung 25*), so dass seine Farbe auf grau und seine Entdeckungszeit auf 2 gesetzt wird. Punkt B liegt auch neben zwei anderen Knoten (C, D), so dass wir der alphabetischen Reihenfolge folgen und Knoten C als nächstes besuchen werden.

Der Besuch von Punkt C (*Abbildung 26*) bringt uns zum Ende eines Zweigs des Baums. Nachdem wir den Knoten grau eingefärbt und seine Entdeckungszeit auf 3 gesetzt haben, stellt der Algorithmus auch fest, dass es keine angrenzenden Knoten zu C gibt. Das bedeutet, dass wir mit der Erkundung des Knotens C fertig sind, und so können wir den Knoten schwarz einfärben und die Endzeit auf 4 setzen. Sie können den Stand unserer Suche an diesem Punkt in *Abbildung 27* sehen.

Da Punkt C das Ende eines Zweiges war, kehren wir nun zu Punkt B zurück und setzen die Erkundung der an B angrenzenden Knoten fort. Der einzige zusätzliche Punkt, den wir von B aus erkunden müssen, ist D, so dass wir nun D besuchen (*Abbildung 28*) und unsere Suche von Punkt D aus fortsetzen können. Punkt D führt uns schnell zu Punkt E (*Abbildung 29*). Normalerweise würden wir diese benachbarten Knoten alphabetisch untersuchen, aber da B bereits grau gefärbt ist, erkennt der Algorithmus, dass er B nicht besuchen sollte, da dies den Algorithmus in eine Schleife bringen würde! Die Untersuchung wird also mit dem nächsten Knoten in der Liste, nämlich F, fortgesetzt (*Abbildung 30*).

Punkt F hat nur einen angrenzenden Punkt, C, aber da C schwarz gefärbt ist, gibt es nichts weiter zu untersuchen, und der Algorithmus hat das Ende eines weiteren Zweiges erreicht. Von hier an sehen Sie in *Abbildung 31* bis *Abbildung 35*, dass sich der Algorithmus zum ersten Knoten zurückarbeitet, indem er die Endzeiten festlegt und die Scheitelpunkte schwarz färbt.

<center><img src="Bilder/Graphen/graphen24.PNG"></center>
<center>Abbildung 24: Kunstruktion des Depth First Suchbaums-10</center>

<center><img src="Bilder/Graphen/graphen25.PNG"></center>
<center>Abbildung 25: Kunstruktion des Depth First Suchbaums-11</center>

<center><img src="Bilder/Graphen/graphen26.PNG"></center>
<center>Abbildung 26: Kunstruktion des Depth First Suchbaums-12</center>

<center><img src="Bilder/Graphen/graphen27.PNG"></center>
<center>Abbildung 27: Kunstruktion des Depth First Suchbaums-13</center>

<center><img src="Bilder/Graphen/graphen28.PNG"></center>
<center>Abbildung 28: Kunstruktion des Depth First Suchbaums-14</center>

<center><img src="Bilder/Graphen/graphen29.PNG"></center>
<center>Abbildung 29: Kunstruktion des Depth First Suchbaums-15</center>

<center><img src="Bilder/Graphen/graphen30.PNG"></center>
<center>Abbildung 30: Kunstruktion des Depth First Suchbaums-16</center>

<center><img src="Bilder/Graphen/graphen31.PNG"></center>
<center>Abbildung 31: Kunstruktion des Depth First Suchbaums-17</center>

<center><img src="Bilder/Graphen/graphen32.PNG"></center>
<center>Abbildung 32: Kunstruktion des Depth First Suchbaums-18</center>

<center><img src="Bilder/Graphen/graphen33.PNG"></center>
<center>Abbildung 33: Kunstruktion des Depth First Suchbaums-19</center>

<center><img src="Bilder/Graphen/graphen34.PNG"></center>
<center>Abbildung 34: Kunstruktion des Depth First Suchbaums-20</center>

<center><img src="Bilder/Graphen/graphen35.PNG"></center>
<center>Abbildung 35: Kunstruktion des Depth First Suchbaums-21</center>

Die Start- und Endzeiten für jeden Knoten zeigen eine Eigenschaft an, die als **Klammereigenschaft** bezeichnet wird. Diese Eigenschaft bedeutet, dass alle Kinder eines bestimmten Knotens im ersten Tiefenbaum eine spätere Entdeckungszeit und eine frühere Endzeit haben als ihre Eltern. Abbildung 26 zeigt den durch den Suchalgorithmus depth first erstellten Baum.

<center><img src="Bilder/Graphen/graphen36.PNG"></center>
<center>Abbildung 36: Der resultierende Depth First Suchbaum</center>

<a id='AnalyseDepthFirstSuche'></a>
## Analyse der Depth First Suche

Die allgemeine Laufzeit für die erste Tiefensuche ist wie folgt. Die Schleifen in `dfs` laufen beide in $O(V)$, wobei das, was in `dfsvisit` geschieht, nicht mitgezählt wird, da sie für jeden Knoten im Graphen einmal ausgeführt werden. In `dfsvisit` wird die Schleife einmal für jede Kante in der Adjazenzliste des aktuellen Vertex ausgeführt. Da `dfsvisit` nur dann rekursiv aufgerufen wird, wenn der Scheitelpunkt weiß ist, wird die Schleife maximal einmal für jede Kante im Graphen oder $O(E)$ ausgeführt. Die Gesamtzeit für die erste Tiefensuche ist also $O(V+E)$.

<a id='TopologischeSortierung'></a>
## Topologische Sortierung

Um zu zeigen, dass Informatiker so gut wie alles in ein Graph-Problem verwandeln können, betrachten wir das schwierige Problem des Aufrührens einer Ladung Pfannkuchen. Das Rezept ist eigentlich ganz einfach: 1 Ei, 1 Tasse Pfannkuchenmischung, 1 Esslöffel Öl und $\frac{3}{4}$ Tassen Milch. Um Pfannkuchen zu machen, muss man die Bratplatte erhitzen, alle Zutaten miteinander vermengen und die Mischung auf eine heiße Platte löffeln. Wenn die Pfannkuchen zu brodeln beginnen, drehen Sie sie um und lassen sie kochen, bis sie am Boden goldbraun sind. Bevor Sie Ihre Pfannkuchen essen, sollten Sie etwas Sirup erhitzen. *Abbildung 37* veranschaulicht diesen Prozess als Graph.

<center><img src="Bilder/Graphen/graphen37.PNG"></center>
<center>Abbildung 37: Die Schritte zur Herstellung von Pfannkuchen</center>

Das Schwierige bei der Herstellung von Pfannkuchen ist zu wissen, was man zuerst tun muss. Wie Sie in *Abbildung 37* sehen können, können Sie damit beginnen, die Grillplatte zu erhitzen oder der Pfannkuchenmischung beliebige Zutaten hinzuzufügen. Um uns bei der Entscheidung zu helfen, in welcher genauen Reihenfolge wir die einzelnen Schritte zur Herstellung unserer Pfannkuchen durchführen sollten, wenden wir uns einem Graph-Algorithmus zu, der als **topologische Sortierung** bezeichnet wird.

Eine topologische Sortierung nimmt einen gerichteten azyklischen Graphen und erzeugt eine lineare Anordnung all seiner Scheitelpunkte, so dass, wenn der Graph $G$ eine Kante $(v,w)$ enthält, der Scheitelpunkt $v vor dem Scheitelpunkt $w in der Anordnung liegt. Gerichtete azyklische Graphen werden in vielen Anwendungen verwendet, um den Vorrang von Ereignissen anzuzeigen. Die Herstellung von Pfannkuchen ist nur ein Beispiel; andere Beispiele sind Software-Projektzeitpläne, Präzedenzdiagramme zur Optimierung von Datenbankabfragen und Multiplikationsmatrizen.

Die topologische Sortierung ist eine einfache, aber nützliche Anpassung einer ersten Tiefensuche. Der Algorithmus für die topologische Sortierung ist wie folgt:

1. Rufen Sie `dfs(g)` für einige Graphen `g` auf. Der Hauptgrund, dfs aufrufen wollen, ist die Berechnung der Endzeiten für jeden der Vertices.
2. Speichern Sie die Eckpunkte in einer Liste in absteigender Reihenfolge der Endzeit.
3. Die geordnete Liste als Ergebnis der topologischen Sortierung zurückgeben.

*Abbildung 38* zeigt die Tiefe des ersten Waldes, der von `dfs` auf dem in *Abbildung 36* gezeigten Graphen zur Pfannkuchenherstellung konstruiert wurde.

<center><img src="Bilder/Graphen/graphen38.PNG"></center>
<center>Abbildung 38: Ergebnis der Depth First Suche im Pfannkuchen Graph</center>

*Abbildung 39* schließlich zeigt die Ergebnisse der Anwendung des topologischen Sortieralgorithmus auf unseren Graphen. Jetzt sind alle Unklarheiten beseitigt, und wir wissen genau, in welcher Reihenfolge wir die Schritte der Pfannkuchenherstellung durchführen müssen.

<center><img src="Bilder/Graphen/graphen39.PNG"></center>
<center>Abbildung 39: Ergebnis der topologischen Sortierung auf gerichtetem azyklischen Graph</center>

<a id='StarkVerbundeneKomponenten'></a>
## Stark verbundene Komponenten

Im weiteren Verlauf dieses Kapitels werden wir unsere Aufmerksamkeit auf einige extrem große Graphen richten. Die Graphen, die wir zur Untersuchung einiger zusätzlicher Algorithmen verwenden werden, sind die Graphen, die durch die Verbindungen zwischen Hosts im Internet und die Links zwischen Webseiten erzeugt werden. Wir werden mit Webseiten beginnen.

Suchmaschinen wie Google und Bing nutzen die Tatsache aus, dass die Seiten im Web einen sehr großen gerichteten Graphen bilden. Um das World Wide Web in einen Graphen zu verwandeln, werden wir eine Seite als einen Scheitelpunkt und die Hyperlinks auf der Seite als Kanten behandeln, die einen Scheitelpunkt mit einem anderen verbinden. *Abbildung 40* zeigt einen sehr kleinen Teil des Graphen, der entsteht, wenn man den Links von einer Seite zur nächsten folgt, beginnend auf der Homepage des Luther-Kollegs für Informatik. Natürlich könnte dieser Graph sehr groß sein, deshalb haben wir ihn auf Websites beschränkt, die nicht mehr als 10 Links von der CS-Homepage entfernt sind.

<center><img src="Bilder/Graphen/graphen40.PNG"></center>
<center>Abbildung 40: Der Graph, erstellt durch Links von der Luther Informatik Homepage</center>

Wenn Sie den Graphen in *Abbildung 40* studieren, könnten Sie einige interessante Beobachtungen machen. Zunächst fällt Ihnen vielleicht auf, dass es sich bei vielen der anderen Websites auf dem Graphen um andere Websites des Luther-Kollegs handelt. Zweitens werden Sie vielleicht feststellen, dass es mehrere Links zu anderen Hochschulen in Iowa gibt. Drittens ist Ihnen vielleicht aufgefallen, dass es mehrere Links zu anderen liberalen Kunsthochschulen gibt. Daraus könnten Sie schließen, dass dem Web eine Struktur zugrunde liegt, die Websites zusammenfasst, die in gewisser Weise ähnlich sind.

Ein Graph-Algorithmus, der helfen kann, Cluster von hochgradig miteinander verbundenen Eckpunkten in einem Graphen zu finden, heißt **Strong Connected Components Algorithmus** (**SCC**). Wir definieren formell eine stark verbundene Komponente $C$ eines Graphen $G$ als die größte Teilmenge von Vertices $C⊂V$, so dass wir für jedes Paar von Vertices $v,w∈C$ einen Pfad von $v$ nach $w$ und einen Pfad von $w$ nach $v$ haben. *Abbildung 37* zeigt einen einfachen Graphen mit drei stark verbundenen Komponenten. Die stark verbundenen Komponenten sind durch die verschiedenen schattierten Bereiche gekennzeichnet.

<center><img src="Bilder/Graphen/graphen41.PNG"></center>
<center>Abbildung 41: Ein gerichteter Graph mit drei stark miteinander verbundenen Komponenten</center>

Sobald die stark verbundenen Komponenten identifiziert worden sind, können wir eine vereinfachte Ansicht des Graphen zeigen, indem wir alle Vertices in einer stark verbundenen Komponente zu einem einzigen größeren Vertex kombinieren. Die vereinfachte Version des Graphen in *Abbildung 41* ist in *Abbildung 42* dargestellt.

<center><img src="Bilder/Graphen/graphen42.PNG"></center>
<center>Abbildung 42: Der reduzierte Graph</center>

Wieder einmal werden wir sehen, dass wir einen sehr leistungsfähigen und effizienten Algorithmus erstellen können, indem wir eine Depth First Suche verwenden. Bevor wir uns mit dem SCC-Hauptalgorithmus befassen, müssen wir uns eine weitere Definition ansehen. Die Transposition eines Graphen $G$ ist definiert als der Graph $G^T$, bei dem alle Kanten im Graphen umgekehrt wurden. Das heißt, wenn es im ursprünglichen Graphen eine gerichtete Kante von Knoten A zu Knoten B gibt, dann enthält GT eine Kante von Knoten B zu Knoten A. *Abbildung 43* und *Abbildung 44* zeigen einen einfachen Graphen und seine Transposition.

<center><img src="Bilder/Graphen/graphen43.PNG"></center>
<center>Abbildung 43: Ein Graph $G$</center>

<center><img src="Bilder/Graphen/graphen44.PNG"></center>
<center>Abbildung 44: Seine Transponierung $G^T$</center>

Schauen Sie sich die Abbildungen noch einmal an. Beachten Sie, dass der Graph in *Abbildung 43* zwei stark miteinander verbundene Komponenten aufweist. Sehen Sie sich nun *Abbildung 44* an. Beachten Sie, dass es die gleichen zwei stark verbundenen Komponenten aufweist.

Wir können nun den Algorithmus zur Berechnung der stark verbundenen Komponenten für einen Graphen beschreiben.

1. Rufen Sie `dfs` für den Graphen G auf, um die Endzeiten für jeden Vertex zu berechnen.
2. Berechnen Sie $G^T$.
3. Rufen Sie `dfs` für den Graphen $G^T$ auf, aber untersuchen Sie in der Hauptschleife der DFS jeden Knoten in abnehmender Reihenfolge der Endzeit.
4. Jeder Baum im Wald, der in Schritt 3 berechnet wurde, ist eine stark verbundene Komponente. Geben Sie die Vertex ids für jeden Vertex in jedem Baum im Wald aus, um die Komponente zu identifizieren.

Verfolgen wir den Ablauf der oben beschriebenen Schritte anhand des Beispielgraphen in *Abbildung 41*. *Abbildung 45* zeigt die Start- und Endzeiten, die für den ursprünglichen Graphen durch den DFS-Algorithmus berechnet wurden. *Abbildung 46* zeigt die Start- und Endzeiten, die für den DFS-Algorithmus im transponierten Graphen berechnet wurden.

<center><img src="Bilder/Graphen/graphen45.PNG"></center>
<center>Abbildung 45: Endzeiten für den ursprünglichen Graphen $G$</center>

<center><img src="Bilder/Graphen/graphen46.PNG"></center>
<center>Abbildung 46: Endzeiten für $G^T$</center>

*Abbildung 47* schließlich zeigt den Wald aus drei Bäumen, der in Schritt 3 des Algorithmus für stark verbundene Komponenten erzeugt wurde. Sie werden feststellen, dass wir Ihnen den Python-Code für den SCC-Algorithmus nicht zur Verfügung stellen, wir lassen das Schreiben dieses Programms als Übung.

<center><img src="Bilder/Graphen/graphen47.PNG"></center>
<center>Abbildung 47: Stark verbundene Komponenten</center>

<a id='ProblemeMitDemKürzestenWeg'></a>
## Probleme mit dem kürzesten Weg

Wenn Sie von einem anderen Ort auf dem Campus im Internet surfen, eine E-Mail senden oder sich an einem Laborcomputer anmelden, wird hinter den Kulissen viel Arbeit geleistet, um die Informationen auf Ihrem Computer auf einen anderen Computer zu übertragen. Die eingehende Untersuchung der Art und Weise, wie Informationen über das Internet von einem Computer auf einen anderen Computer übertragen werden, ist das Hauptthema für einen Kurs über Computernetzwerke. Wir werden jedoch darüber sprechen, wie das Internet gerade genug funktioniert, um einen anderen sehr wichtigen Graph-Algorithmus zu verstehen.

<center><img src="Bilder/Graphen/graphen48.PNG"></center>
<center>Abbildung 48: Überblick über die Konnektivität im Internet</center>

*Abbildung 48* zeigt Ihnen einen Überblick auf hoher Ebene, wie die Kommunikation im Internet funktioniert. Wenn Sie Ihren Browser benutzen, um eine Webseite von einem Server anzufordern, muss die Anfrage über Ihr lokales Netzwerk und über einen Router ins Internet geleitet werden. Die Anfrage reist über das Internet und kommt schließlich bei einem Router für das lokale Netzwerk an, in dem sich der Server befindet. Die von Ihnen angeforderte Webseite reist dann über dieselben Router zurück, um zu Ihrem Browser zu gelangen. Innerhalb der Wolke mit der Bezeichnung "Internet" in *Abbildung 48* befinden sich weitere Router. Die Aufgabe all dieser Router ist es, zusammenzuarbeiten, um Ihre Informationen von Ort zu Ort zu bringen. Sie können sehen, dass es viele Router für Sie selbst gibt, wenn Ihr Computer den Befehl *traceroute* unterstützt. Der folgende Text zeigt die Ausgabe des *Traceroute*-Befehls, der veranschaulicht, dass es 13 Router zwischen dem Webserver am Luther College und dem Mailserver an der University of Minnesota gibt.

```
1  192.203.196.1
2  hilda.luther.edu (216.159.75.1)
3  ICN-Luther-Ether.icn.state.ia.us (207.165.237.137)
4  ICN-ISP-1.icn.state.ia.us (209.56.255.1)
5  p3-0.hsa1.chi1.bbnplanet.net (4.24.202.13)
6  ae-1-54.bbr2.Chicago1.Level3.net (4.68.101.97)
7  so-3-0-0.mpls2.Minneapolis1.Level3.net (64.159.4.214)
8  ge-3-0.hsa2.Minneapolis1.Level3.net (4.68.112.18)
9  p1-0.minnesota.bbnplanet.net (4.24.226.74)
10  TelecomB-BR-01-V4002.ggnet.umn.edu (192.42.152.37)
11  TelecomB-BN-01-Vlan-3000.ggnet.umn.edu (128.101.58.1)
12  TelecomB-CN-01-Vlan-710.ggnet.umn.edu (128.101.80.158)
13  baldrick.cs.umn.edu (128.101.80.129)(N!)  88.631 ms (N!)


Routers from One Host to the Next over the Internet
```

Jeder Router im Internet ist mit einem oder mehreren anderen Routern verbunden. Wenn Sie also den Befehl `traceroute` zu unterschiedlichen Tageszeiten ausführen, werden Sie wahrscheinlich feststellen, dass Ihre Informationen zu unterschiedlichen Zeiten durch verschiedene Router fließen. Das liegt daran, dass mit jeder Verbindung zwischen einem Routerpaar Kosten verbunden sind, die vom Verkehrsaufkommen, der Tageszeit und vielen anderen Faktoren abhängen. Zu diesem Zeitpunkt wird es Sie nicht mehr überraschen zu erfahren, dass wir das Routernetz als Graph mit gewichteten Kanten darstellen können.

<center><img src="Bilder/Graphen/graphen49.PNG"></center>
<center>Abbildung 49: Verbindungen und Gewichte zwischen Routern im Internet</center>

*Abbildung 49* zeigt ein kleines Beispiel für einen gewichteten Graphen, der die Zusammenschaltung von Routern im Internet darstellt. Das Problem, das wir lösen wollen, besteht darin, den Pfad mit dem geringsten Gesamtgewicht zu finden, auf dem eine bestimmte Nachricht weitergeleitet werden kann. Dieses Problem sollte Ihnen bekannt vorkommen, denn es ähnelt dem Problem, das wir mit einer ersten Breitensuche gelöst haben, mit der Ausnahme, dass es hier um das Gesamtgewicht des Pfades und nicht um die Anzahl der Sprünge im Pfad geht. Es ist zu beachten, dass das Problem dasselbe ist, wenn alle Gewichte gleich sind.

<a id='DerDijkstraAlgorithmus'></a>
## Der Dijkstra Algorithmus

Der Algorithmus, den wir verwenden werden, um den kürzesten Weg zu bestimmen, heißt "Dijkstra-Algorithmus". Der Dijkstra-Algorithmus ist ein iterativer Algorithmus, der uns den kürzesten Weg von einem bestimmten Startknoten zu allen anderen Knoten im Graphen liefert. Auch dies ist ähnlich wie die Ergebnisse einer ersten Breitensuche.

Um die Gesamtkosten vom Startknoten bis zu jedem Ziel zu verfolgen, verwenden wir die Instanzvariable `dist` in der Vertex-Klasse. Die `dist` Instanzvariable enthält das aktuelle Gesamtgewicht des kleinsten Gewichtungspfades vom Start bis zum fraglichen Vertex. Der Algorithmus iteriert einmal für jeden Vertex im Graphen; die Reihenfolge, in der wir über die Vertices iterieren, wird jedoch durch eine Prioritätswarteschlange gesteuert. Der Wert, der zur Bestimmung der Reihenfolge der Objekte in der Prioritätswarteschlange verwendet wird, ist `dist`. Wenn ein Vertex zum ersten Mal erzeugt wird, wird `dist` auf eine sehr große Zahl gesetzt. Theoretisch würde man `dist` auf unendlich setzen, aber in der Praxis setzen wir es einfach auf eine Zahl, die größer ist als jeder reale Abstand, den wir bei dem zu lösenden Problem hätten.

Der Code für den Dijkstra-Algorithmus ist in **Auflistung 11** dargestellt. Wenn der Algorithmus beendet ist, sind die Abstände korrekt eingestellt, ebenso wie die Vorgänger-Links für jeden Knoten im Graphen.

**Auflistung 11**
```python
from pythonds.graphs import PriorityQueue, Graph, Vertex
def dijkstra(aGraph,start):
    pq = PriorityQueue()
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(),v) for v in aGraph])
    while not pq.isEmpty():
        currentVert = pq.delMin()
        for nextVert in currentVert.getConnections():
            newDist = currentVert.getDistance() \
                    + currentVert.getWeight(nextVert)
            if newDist < nextVert.getDistance():
                nextVert.setDistance( newDist )
                nextVert.setPred(currentVert)
                pq.decreaseKey(nextVert,newDist)
```

Der Algorithmus von Dijkstra verwendet eine Prioritätswarteschlange. Sie erinnern sich vielleicht daran, dass eine Prioritätswarteschlange auf dem Heap basiert, den wir im Kapitel "Bäume" implementiert haben. Es gibt einige Unterschiede zwischen dieser einfachen Implementierung und der Implementierung, die wir für den Algorithmus von Dijkstra verwenden. Erstens speichert die `PriorityQueue`-Klasse Tupel von Schlüssel- und Wertepaaren. Dies ist für den Algorithmus von Dijkstra wichtig, da der Schlüssel in der Prioritätswarteschlange mit dem Schlüssel des Vertex im Graphen übereinstimmen muss. Zweitens wird der Wert für die Entscheidung über die Priorität und damit die Position des Schlüssels in der Prioritätswarteschlange verwendet. In dieser Implementierung verwenden wir den Abstand zum Scheitelpunkt als Priorität, denn wie wir sehen werden, wenn wir den nächsten Scheitelpunkt erforschen, wollen wir immer den Scheitelpunkt mit dem geringsten Abstand erforschen. Der zweite Unterschied ist die Hinzufügung der `decreaseKey`-Methode. Wie Sie sehen können, wird diese Methode verwendet, wenn der Abstand zu einem bereits in der Warteschlange befindlichen Scheitelpunkt verringert wird und somit dieser Scheitelpunkt nach vorne in die Warteschlange verschoben wird.

Gehen wir eine Anwendung des Dijkstra-Algorithmus Punkt für Punkt durch, wobei wir uns an der folgenden Abbildung orientieren. Wir beginnen mit dem Scheitelpunkt u. Die drei an u angrenzenden Vertices sind v,w und x. Da die Anfangsabstände zu v,w und x alle mit `sys.maxint` initialisiert sind, sind die neuen Kosten, um sie durch den Startknoten zu erreichen, alle ihre direkten Kosten. Wir aktualisieren also die Kosten für jeden dieser drei Knoten. Wir setzen auch den Vorgänger für jeden Knoten auf u, und wir fügen jeden Knoten zur Prioritätswarteschlange hinzu. Wir verwenden die Entfernung als Schlüssel für die Prioritätswarteschlange. Der Zustand des Algorithmus ist in *Abbildung 51* dargestellt.

In der nächsten Iteration der `while`-Schleife untersuchen wir die Vertices, die an $x$ angrenzen. Der Vertex $x$ ist der nächste, weil er die geringsten Gesamtkosten hat und sich daher mit Blasen an den Anfang der Prioritätswarteschlange vorgearbeitet hat. Bei $x $schauen wir uns seine Nachbarn $u,v,w$ und $y$ an. Für jeden benachbarten Scheitelpunkt prüfen wir, ob der Abstand zu diesem Scheitelpunkt durch $x$ kleiner als der zuvor bekannte Abstand ist. Offensichtlich ist dies für y der Fall, da sein Abstand `sys.maxint` war. Dies ist nicht der Fall für $u$ oder $v$, da ihre Abstände 0 bzw. 2 betragen. Nun lernen wir jedoch, dass der Abstand zu $w$ kleiner ist, wenn wir durch $x$ gehen als von $u$ direkt zu $w$. Da dies der Fall ist, aktualisieren wir $w$ mit einem neuen Abstand und ändern den Vorgänger für $w$ von $u$ zu $x$. Siehe *Abbildung 52* für den Zustand aller Scheitelpunkte.

Der nächste Schritt besteht darin, die $v$ benachbarten Eckpunkte zu betrachten (siehe *Abbildung 53*). Bei Knoten y (siehe *Abbildung 54*) stellen wir fest, dass es billiger ist, sowohl zu $w$ als auch zu $z$ zu gelangen, also passen wir die Abstände und Vorgängerlinks entsprechend an. Schließlich überprüfen wir die Knoten $w$ und $z$ (siehe *Abbildung 55* und *Abbildung 56*). Es werden jedoch keine zusätzlichen Änderungen gefunden, so dass die Prioritätswarteschlange leer ist und der Dijkstra-Algorithmus beendet wird.

<center><img src="Bilder/Graphen/graphen50.PNG"></center>
<center>Abbildung 50: Der Algorithmus von Dijkstra</center>

<center><img src="Bilder/Graphen/graphen51.PNG"></center>
<center>Abbildung 51: Der Algorithmus von Dijkstra</center>

<center><img src="Bilder/Graphen/graphen52.PNG"></center>
<center>Abbildung 52: Der Algorithmus von Dijkstra</center>

<center><img src="Bilder/Graphen/graphen53.PNG"></center>
<center>Abbildung 53: Der Algorithmus von Dijkstra</center>

<center><img src="Bilder/Graphen/graphen54.PNG"></center>
<center>Abbildung 54: Der Algorithmus von Dijkstra</center>

<center><img src="Bilder/Graphen/graphen55.PNG"></center>
<center>Abbildung 55: Der Algorithmus von Dijkstra</center>

Es ist wichtig zu beachten, dass der Algorithmus von Dijkstra nur funktioniert, wenn die Gewichte alle positiv sind. Sie sollten sich selbst davon überzeugen, dass der Algorithmus niemals aus dem Graphen austreten würde, wenn Sie eine negative Gewichtung an einer der Kanten des Graphen einführen.

Wir werden feststellen, dass für die Weiterleitung von Nachrichten durch das Internet andere Algorithmen verwendet werden, um den kürzesten Weg zu finden. Eines der Probleme bei der Verwendung des Dijkstra-Algorithmus im Internet besteht darin, dass Sie eine vollständige Darstellung des Graphen haben müssen, damit der Algorithmus ausgeführt werden kann. Dies hat zur Folge, dass jeder Router über eine vollständige Abbildung aller Router im Internet verfügt. In der Praxis ist dies nicht der Fall, und andere Varianten des Algorithmus ermöglichen es jedem Router, den Graphen im laufenden Betrieb zu entdecken. Ein solcher Algorithmus, über den Sie vielleicht etwas lesen möchten, wird als "Entfernungsvektor"-Routing-Algorithmus bezeichnet.

<a id='AnalyseDijkstraAlgorithmus'></a>
## Analyse des Dijkstra Algorithmus

Schließlich wollen wir uns die Laufzeit des Dijkstra-Algorithmus ansehen. Zunächst stellen wir fest, dass der Aufbau der Prioritätswarteschlange $O(V)$-Zeit benötigt, da wir anfangs jeden Knoten im Graphen zur Prioritätswarteschlange hinzufügen. Sobald die Warteschlange aufgebaut ist, wird die while-Schleife für jeden Knoten einmal ausgeführt, da die Knoten alle am Anfang hinzugefügt und erst danach entfernt werden. Innerhalb dieser Schleife benötigt jeder Aufruf von delMin $O(log V)$-Zeit. Zusammengenommen nehmen dieser Teil der Schleife und die Aufrufe an delMin $O(V log(V))$ in Anspruch. Die for-Schleife wird für jede Kante im Graphen einmal ausgeführt, und innerhalb der for-Schleife dauert der Aufruf von decreaseKey die Zeit O$(E log(V))$. Die kombinierte Laufzeit ist also $O((V+E)log(V))$.

<a id='DerA*Algorithmus'></a>
## Der A* Algorithmus
Eine Verbesserung des eben gelernten Dijkstra Algorithmus ist der **A* Algorithmus** (auch A-Stern-Algorithmus genannt). Da der A* auf dem Dijkstra Algorithmus aufbaut, ist er ebenfalls ein iterativer Algorithmus, der uns den kürzesten Weg von einem bestimmten Startknoten zu allen anderen Knoten im Graphen liefert. Dazu verwendet er nicht nur die Abstände zwischen den einzelnen Knoten, sondern zusätzlich noch eine Heuristik Funktion, um diese Abstände zu gewichten.

Um den kürzesten Weg zum Zielknoten zu finden, untersucht der A* Algorithmus immer den Knoten zuerst, der wahrscheinlich schnell zum Ziel führt. Dazu wird den bekannten Knoten *n* ein Wert $f(n)$ zugeordnet, der eine Abschätzung angibt, wie lang der Pfad vom Start zum Ziel unter Verwendung des betrachteten Knotens im günstigsten Fall ist. Der Knoten, der den niedrigsten f-Wert hat, wird als nächstes betrachtet.

Den Wert f(n) berechnet man folgendermaßen:
$$f(n) = g(n) + h(n)$$
$g(n)$ beschreibt dabei die Kosten vom vom Startknoten aus bis zum aktuell betrachteten Knoten *n*.    
$h(n)$ beschreibt die geschätzten Kosten vom aktuell betrachteten Knoten *n* bis zum Zielknoten. Diesen geschätzten Wert kann man der Heuristik entnehmen.

**Vorgehen**     
Für den A* Algorithmus erstellt man zuerst zwei Listen, eine OPEN-Liste und eine CLOSES-Liste. Diese besteht jeweils aus 3 Spalten. In die erste Spalte der OPEN-Liste wird die Bezeichnung des Knoten geschrieben, in der zweiten Splate berechnet man den Wert für f(n) und in die dritte Spalte schreibt man den Vorgängerknoten. Die CLOSE-Lsite ist so ähnlich aufgebaut, nur wird hier in der zweiten Spalte der Wert für g(n) angegeben.    

Man startet den A* Algorithmus indem man den Startknoten in die OPEN-Liste schreibt.   
Danach werden alle Knoten, die direkt vom Startknoten aus erreicht werden können, die direkten Nachbarn, in die OPEN-Liste eingetragen. Jetzt vergleicht man den Wert für f(n) für jeden Knoten. Der Knoten mit dem niedrigsten Wert wird aus der OPEN-Liste gestrichen und in die COLSE-Liste eingetragen.     
Von nun an wiederholt sich der Vorgang, bis der Zielknoten in die CLOSE-Liste eingetragen wurde.    
1. Knoten mit den kleinsten Kosten finden.   
2. Alle direkten Nachbarn von diesem Knoten in die OPEN-Liste schreiben. Überprüfen, ob Knoten die schon in der OPEN-Liste stehen über den neuen Knoten schneller erreicht werden können. Knoten ggf. aktualisieren.
3. Knoten mit den kleinsten Kosten aus der OPEN-Liste streichen und in die CLOSE-Liste eintragen.

**Beispiel**  
An folgendem Beispiel gehen wir den A* Algorithmus einmal durch. Wir wollen den kürzesten Weg von Knoten 1 zu Knoten 7 finden.   
![Beispiel A*](Bilder/Graphen/AStern_Beispiel.png)   

| Knoten    | 1  | 2  | 3  | 4  | 5  | 6  | 7 | 8  |
|-----------|----|----|----|----|----|----|---|----|
| Heuristik | 30 | 20 | 15 | 10 | 10 | 40 | 0 | 70 |       

Startknoten in die OPEN-Liste schreiben 

| OPEN   | f(n)    | V |
|--------|---------|---|
| 1      | 0+30=30 | / |
|        |         |   |
|        |         |   |
|        |         |   |
|        |         |   |
|        |         |   |
|        |         |   |

| CLOSE | g(n) | V |
|-|-|-|
|  |  |  |
|  |  |  |
|  |  |  |
|  |  |  |
|  |  |  |

1. Aktuell kleinstes f(n) suchen -> Knoten 1    
2. Alle direkten Nachbarn von Knoten 1 in die OPEN-Liste schreiben    
    Kann man einen der Knoten in der OPEN-Liste aktualisieren? -> Nein     
3. Knoten 1 aus der OPEN-Liste steichen und in die CLOSE-Liste schreiben

| OPEN | f(n) | V |
|-|-|-|
| ~~1~~ | ~~0+30=30~~ | ~~/~~ |
| 2 | 10+20=30 | 1 |
| 3 | 20+15=35 | 1 |
|  |  |  |
|  |  |  |
|  |  |  |
|  |  |  |

| CLOSE | g(n) | V |
|-|-|-|
| 1 | 0 | / |
|  |  |  |
|  |  |  |
|  |  |  |
|  |  |  |

1. Aktuell kleinstes f(n) suchen -> Knoten 2    
2. Alle direkten Nachbarn von Knoten 2 in die OPEN-Liste schreiben    
    Kann man einen der Knoten in der OPEN-Liste aktualisieren? -> Nein     
3. Knoten 2 aus der OPEN-Liste steichen und in die CLOSE-Liste schreiben

| OPEN | f(n) | V |
|-|-|-|
| ~~1~~ | ~~0+30=30~~ | ~~/~~ |
| ~~2~~ | ~~10+20=30~~ | ~~1~~ |
| 3 | 20+15=35 | 1 |
| 6 | 30+40=70 | 2 |
|  |  |  |
|  |  |  |
|  |  |  |

| CLOSE | g(n) | V |
|-|-|-|
| 1 | 0 | / |
| 2 | 10 | 1 |
|  |  |  |
|  |  |  |
|  |  |  |

1. Aktuell kleinstes f(n) suchen -> Knoten 3    
2. Alle direkten Nachbarn von Knoten 3 in die OPEN-Liste schreiben    
    Kann man einen der Knoten in der OPEN-Liste aktualisieren? -> Nein     
3. Knoten 3 aus der OPEN-Liste steichen und in die CLOSE-Liste schreiben

| OPEN | f(n) | V |
|-|-|-|
| ~~1~~ | ~~0+30=30~~ | ~~/~~ |
| ~~2~~ | ~~10+20=30~~ | ~~1~~ |
| ~~3~~ | ~~20+15=35~~ | ~~1~~ |
| 6 | 30+40=70 | 2 |
| 4 | 25+10=35 | 3 |
| 7 | 50+0=50 | 3 |
|  |  |  |

| CLOSE | g(n) | V |
|-|-|-|
| 1 | 0 | / |
| 2 | 10 | 1 |
| 3 | 15 | 2 |
|  |  |  |
|  |  |  |


1. Aktuell kleinstes f(n) suchen -> Knoten 4    
2. Alle direkten Nachbarn von Knoten 4 in die OPEN-Liste schreiben    
    Kann man einen der Knoten in der OPEN-Liste aktualisieren? -> Ja, Knoten 7     
3. Knoten 4 aus der OPEN-Liste steichen und in die CLOSE-Liste schreiben

| OPEN | f(n) | V |
|-|-|-|
| ~~1~~ | ~~0+30=30~~ | ~~/~~ |
| ~~2~~ | ~~10+20=30~~ | ~~1~~ |
| ~~3~~ | ~~20+15=35~~ | ~~1~~ |
| 6 | 30+40=70 | 2 |
| ~~4~~ | ~~25+10=35~~ | ~~3~~ |
| 7 | ~~50+0=50~~ 40+0=40 | ~~3~~ 4 |
| 5 | 30+10=40 | 4 |

| CLOSE | g(n) | V |
|-|-|-|
| 1 | 0 | / |
| 2 | 10 | 1 |
| 3 | 15 | 2 |
| 4 | 20 | 3 |
|  |  |  |

1. Aktuell kleinstes f(n) suchen -> Knoten 7 und Knoten 5 sind gleich groß -> Knoten 7 kam zuerst in die Liste    
2. Alle direkten Nachbarn von Knoten 7 in die OPEN-Liste schreiben    
    Kann man einen der Knoten in der OPEN-Liste aktualisieren? -> Nein     
3. Knoten 7 aus der OPEN-Liste steichen und in die CLOSE-Liste schreiben

| OPEN | f(n) | V |
|-|-|-|
| ~~1~~ | ~~0+30=30~~ | ~~/~~ |
| ~~2~~ | ~~10+20=30~~ | ~~1~~ |
| ~~3~~ | ~~20+15=35~~ | ~~1~~ |
| 6 | 30+40=70 | 2 |
| ~~4~~ | ~~25+10=35~~ | ~~3~~ |
| ~~7~~ | ~~40+0=40~~ | ~~4~~ |
| 5 | 30+10=40 | 4 |

| CLOSE | g(n) | V |
|-|-|-|
| 1 | 0 | / |
| 2 | 10 | 1 |
| 3 | 15 | 2 |
| 4 | 20 | 3 |
| 7 | 40 | 4 |

Der Zielknoten ist in der CLOSE-Liste.    
Weg: 1 - 2 - 3 - 4 - 7

<a id='PrimSpannbaum-Algorithmus'></a>
## Prim's Spannbaum-Algorithmus

Für unseren letzten Graph-Algorithmus betrachten wir ein Problem, mit dem sich die Entwickler von Online-Spielen und Internet-Radioanbietern konfrontiert sehen. Das Problem besteht darin, dass sie eine Information effizient an alle und jeden, der zuhören könnte, übertragen wollen. Das ist beim Spielen wichtig, damit alle Spieler die aktuelle Position jedes anderen Spielers kennen. Beim Internet-Radio ist dies wichtig, damit alle Zuhörer, die eingeschaltet sind, alle Daten erhalten, die sie brauchen, um den Song, den sie hören, zu rekonstruieren. *Abbildung 56* veranschaulicht das Übertragungsproblem.

<center><img src="Bilder/Graphen/graphen56.PNG"></center>
<center>Abbildung 56: Das Broadcast-Problem</center>

Es gibt einige Brute-Force-Lösungen für dieses Problem, also schauen wir uns diese zunächst einmal an, um das Rundfunkproblem besser zu verstehen. Das wird Ihnen auch helfen, die Lösung zu verstehen, die wir vorschlagen werden, wenn wir fertig sind. Zu Beginn hat der Moderator der Sendung einige Informationen, die die Zuhörer alle empfangen müssen. Die einfachste Lösung besteht darin, dass der Moderator eine Liste aller Zuhörer führt und an jeden einzelnen Zuhörer eine individuelle Nachricht sendet. In *Abbildung 56* zeigen wir ein kleines Netzwerk mit einem Rundfunkveranstalter und einigen Zuhörern. Bei diesem ersten Ansatz würden von jeder Nachricht vier Kopien gesendet. Unter der Annahme, dass der kostengünstigste Weg verwendet wird, wollen wir sehen, wie oft jeder Router die gleiche Nachricht bearbeiten würde.

Alle Nachrichten des Broadcasters durchlaufen Router A, so dass A alle vier Kopien jeder Nachricht sieht. Router C sieht nur eine Kopie jeder Nachricht für seine Zuhörer. Die Router B und D würden jedoch drei Kopien jeder Nachricht sehen, da die Router B und D auf dem günstigsten Weg für die Zuhörer 1, 2 und 3 liegen. Wenn man bedenkt, dass der Sende-Host für eine Radiosendung jede Sekunde Hunderte von Nachrichten senden muss, ist das eine Menge zusätzlicher Datenverkehr.

Eine Brute-Force-Lösung besteht darin, dass der Moderator eine einzige Kopie der Rundfunkmeldung sendet und die Router die Dinge regeln lassen. In diesem Fall ist die einfachste Lösung eine Strategie namens **unkontrollierte Überflutung**. Die Überflutungsstrategie funktioniert wie folgt. Jede Nachricht beginnt mit einem Time-to-Live (`ttl`)-Wert, der auf eine Zahl eingestellt ist, die größer oder gleich der Anzahl der Flanken zwischen dem Sende-Host und dem am weitesten entfernten Zuhörer ist. Jeder Router erhält eine Kopie der Nachricht und leitet die Nachricht an alle seine Nachbarrouter weiter. Wenn die Nachricht weitergeleitet wird, wird der `ttl`-Wert verringert. Jeder Router sendet weiterhin Kopien der Nachricht an alle seine Nachbarn, bis der `ttl`-Wert 0 erreicht. Man kann sich leicht davon überzeugen, dass eine unkontrollierte Überflutung viel mehr unnötige Nachrichten erzeugt als unsere erste Strategie.

Die Lösung dieses Problems liegt in der Konstruktion eines **minimalen Spannbaumes**. Formal definieren wir den minimalen überspannenden Baum $T$ für einen Graphen $G=(V,E)$ wie folgt. $T$ ist eine azyklische Teilmenge von $E$, die alle Eckpunkte in $V$ verbindet. Die Summe der Gewichte der Kanten in $T$ wird minimiert.

*Abbildung 57* zeigt eine vereinfachte Version des Broadcast-Graphen und hebt die Kanten hervor, die einen minimalen Spannbaum für den Graphen bilden. Um nun unser Broadcast-Problem zu lösen, sendet der Broadcast-Host einfach eine einzige Kopie der Broadcast-Nachricht in das Netzwerk. Jeder Router leitet die Nachricht an jeden Nachbarn weiter, der Teil des Spannbaums ist, mit Ausnahme des Nachbarn, der ihm gerade die Nachricht gesendet hat. In diesem Beispiel leitet A die Nachricht an B weiter. B leitet die Nachricht an D und C weiter. D leitet die Nachricht an E weiter, das sie an F weiterleitet, das sie an G weiterleitet. Kein Router sieht mehr als eine Kopie einer Nachricht, und alle interessierten Zuhörer sehen eine Kopie der Nachricht.

<center><img src="Bilder/Graphen/graphen57.PNG"></center>
<center>Abbildung 57: Minimaler Spannbaum für den Broadcast Graph</center>

Der Algorithmus, den wir zur Lösung dieses Problems verwenden werden, heißt Prim-Algorithmus. Der Prim-Algorithmus gehört zu einer Familie von Algorithmen, die als "gierige Algorithmen" bezeichnet werden, weil wir bei jedem Schritt den billigsten nächsten Schritt wählen werden. In diesem Fall besteht der billigste nächste Schritt darin, der Kante mit dem geringsten Gewicht zu folgen. Unser letzter Schritt ist die Entwicklung des Prim-Algorithmus.

Die Grundidee bei der Konstruktion eines überspannenden Baumes lautet wie folgt:
```
While T is not yet a spanning tree
   Find an edge that is safe to add to the tree
   Add the new edge to T
```

Der Trick liegt in dem Schritt, der uns anleitet, "einen sicheren Rand zu finden". Wir definieren eine sichere Kante als jede Kante, die einen Scheitelpunkt, der sich im Spannbaum befindet, mit einem Scheitelpunkt verbindet, der sich nicht im Spannbaum befindet. Auf diese Weise wird sichergestellt, dass der Baum immer ein Baum bleibt und daher keine Zyklen aufweist.

Der Python-Code zur Implementierung des Prim-Algorithmus ist in *Auflistung 12* dargestellt. Der Algorithmus von Prim ist dem Algorithmus von Dijkstra insofern ähnlich, als beide eine Prioritätswarteschlange verwenden, um den nächsten Knoten auszuwählen, der dem wachsenden Graphen hinzugefügt werden soll.

**Auflistung 12**
```python
from pythonds.graphs import PriorityQueue, Graph, Vertex

def prim(G,start):
    pq = PriorityQueue()
    for v in G:
        v.setDistance(sys.maxsize)
        v.setPred(None)
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(),v) for v in G])
    while not pq.isEmpty():
            currentVert = pq.delMin()
            for nextVert in currentVert.getConnections():
                newCost = currentVert.getWeight(nextVert)
                if nextVert in pq and newCost<nextVert.getDistance():
                    nextVert.setPred(currentVert)
                    nextVert.setDistance(newCost)
                    pq.decreaseKey(nextVert,newCost)
```

Die folgende Abbildung (*Abbildung 58* bis *Abbildung 59*) zeigt den Algorithmus in unserem Beispielbaum. Wir beginnen mit dem Startpunkt als A. Die Abstände zu allen anderen Punkten sind auf unendlich initialisiert. Wenn wir die Nachbarn von A betrachten, können wir die Abstände zu zwei der zusätzlichen Eckpunkte B und C aktualisieren, da die Abstände zu B und C bis A kleiner als unendlich sind. Dadurch werden B und C an den Anfang der Prioritätswarteschlange verschoben. Aktualisieren Sie die Vorgänger-Links für B und C, indem Sie sie so einstellen, dass sie auf A zeigen. Es ist wichtig zu beachten, dass wir B oder C noch nicht formell zum überspannenden Baum hinzugefügt haben. Ein Knoten gilt erst dann als Teil des überspannenden Baums, wenn er aus der Prioritätswarteschlange entfernt wird.

Da B den kleinsten Abstand hat, sehen wir uns B als nächstes an. Wenn wir die Nachbarn von B untersuchen, sehen wir, dass D und E aktualisiert werden können. Sowohl D als auch E erhalten neue Abstandswerte, und ihre Vorgängerverbindungen werden aktualisiert. Wenn wir zum nächsten Knoten in der Prioritätswarteschlange weitergehen, finden wir C. Der einzige benachbarte Knoten C, der sich noch in der Prioritätswarteschlange befindet, ist F, so dass wir den Abstand zu F aktualisieren und die Position von F in der Prioritätswarteschlange anpassen können.

Nun untersuchen wir die an den Knoten D angrenzenden Scheitelpunkte. Wir stellen fest, dass wir E aktualisieren und den Abstand zu E von 6 auf 4 reduzieren können. Wenn wir dies tun, ändern wir die Vorgängerverbindung auf E so, dass sie wieder auf D zeigt, und bereiten sie so darauf vor, in den überspannenden Baum verpflanzt zu werden, aber an einer anderen Stelle. Der Rest des Algorithmus verläuft wie erwartet und fügt jeden neuen Knoten zum Baum hinzu.

<center><img src="Bilder/Graphen/graphen58.PNG"></center>
<center>Abbildung 58: Algorithmus von Tracing Prim</center>

<center><img src="Bilder/Graphen/graphen59.PNG"></center>
<center>Abbildung 59: Algorithmus von Tracing Prim</center>

<center><img src="Bilder/Graphen/graphen60.PNG"></center>
<center>Abbildung 60: Algorithmus von Tracing Prim</center>

<center><img src="Bilder/Graphen/graphen61.PNG"></center>
<center>Abbildung 61: Algorithmus von Tracing Prim</center>

<center><img src="Bilder/Graphen/graphen62.PNG"></center>
<center>Abbildung 62: Algorithmus von Tracing Prim</center>

<center><img src="Bilder/Graphen/graphen63.PNG"></center>
<center>Abbildung 63: Algorithmus von Tracing Prim</center>

<center><img src="Bilder/Graphen/graphen64.PNG"></center>
<center>Abbildung 64: Algorithmus von Tracing Prim</center>

<a id='Zusammenfassung'></a>
## Zusammenfassung

In diesem Kapitel haben wir uns mit dem abstrakten Datentyp Graph und einigen Implementierungen eines Graphen befasst. Ein Graph ermöglicht es uns, viele Probleme zu lösen, vorausgesetzt, wir können das ursprüngliche Problem in etwas umwandeln, das durch einen Graphen dargestellt werden kann. Insbesondere haben wir gesehen, dass Graphen nützlich sind, um Probleme in den folgenden allgemeinen Bereichen zu lösen.

* Breadth First Suche für den ungewichteten kürzesten Weg.
* Dijkstra's Algorithmus für den gewichteten kürzesten Weg.
* Depth First Suche nach der Erforschung von Graphen.
* Stark verbundene Komponenten zur Vereinfachung eines Graphen.
* Topologische Sortierung für Ordnungsaufgaben.
* Minimale Spannbäume für das Senden von Nachrichten.

<a id='Diskussion'></a>
## Fragen zur Diskussion

1. Zeichnen Sie den Graphen entsprechend der folgenden Adjazenzmatrix.
<center><img src="Bilder/Graphen/graphen65.PNG"></center>
2. Zeichnen Sie den Graphen, der der folgenden Liste von Kanten entspricht.

| **von** | **zu** | **Kosten** |
|:---------:|:-----------:|:----------:|
|    1    |    2   |   10  |
|    1    |    3   |   15   |
|    1    |    6   |   5   |
|    2    |    3   |   7   |
|    3    |    4   |   7   |
|    3    |    6   |   10   |
|    4    |    5   |   7   |
|    6    |    4   |   5   |
|    5    |    6   |   13   |

3. Ignorieren Sie die Gewichte und führen Sie eine Breadth First Suche auf dem Graphen aus der vorherigen Frage durch:

In [ ]:
# TODO

4. Was ist die Big-O-Laufzeit der `buildGraph`-Funktion?

In [ ]:
hide_me
import ipywidgets as widgets

answers1 = widgets.RadioButtons(
    options=['O(n)',
             'O(n^2)',
             'O(1)',
             'O(n^3)'],
    disabled=False
)
button1 = widgets.Button(
    description='Überprüfen',
    disabled=False,
    tooltip='Click me',
    icon='check'
)
def button1_eventhandler(obj):
    if answers1.value == 'O(n^2)':
        print("Antwort 1 ist richtig.")
    else:
        print("Antwort 1 ist falsch.")
    
button1.on_click(button1_eventhandler)

display(answers1)

display(button1)

5. Leiten Sie die Big-O-Laufzeit für den topologischen Sortieralgorithmus ab.
6. Leiten Sie die Big-O-Laufzeit für den Algorithmus für stark verbundene Komponenten ab.
7. Zeigen Sie jeden Schritt bei der Anwendung des Dijkstra-Algorithmus auf den oben gezeigten Graphen.
8. Finden Sie mit Hilfe des Algorithmus von Prim den Baum mit der minimalen Gewichtsspanne für den oben gezeigten Graphen.
9. Zeichnen Sie einen Abhängigkeitsgraphen, der die zum Versenden einer E-Mail erforderlichen Schritte veranschaulicht. Führen Sie eine topologische Sortierung Ihres Graphen durch.
10. Leiten Sie einen Ausdruck für die Basis des Exponenten ab, der verwendet wird, um die Laufzeit der Rittertour auszudrücken.
11. Erklären Sie, warum der allgemeine DFS-Algorithmus für die Lösung nicht geeignet ist das Rittertour-Problem.
12. Was ist die Big-O-Laufzeit für Prims minimalen Spannbaum?

In [ ]:
hide_me
import ipywidgets as widgets

answers1 = widgets.RadioButtons(
    options=['O(1)',
             'O(n^3)',
             'O(n)',
             'O(n^2)'],
    disabled=False
)
button1 = widgets.Button(
    description='Überprüfen',
    disabled=False,
    tooltip='Click me',
    icon='check'
)
def button1_eventhandler(obj):
    if answers1.value == 'O(n^2)':
        print("Antwort 1 ist richtig.")
    else:
        print("Antwort 1 ist falsch.")
    
button1.on_click(button1_eventhandler)

display(answers1)

display(button1)

<a id='Programmieraufgaben'></a>
## Programmieraufgaben
1. Modifizieren Sie die  depth first Suchfunktion, um eine topologische Sortierung zu erzeugen.
2. Ändern Sie die erste Tiefensuche, um stark verbundene Komponenten zu erzeugen.
3. Schreiben Sie die `Transpose`-Methode für die `Graph`-Klasse.
4. Unter Verwendung der Breitensuche schreiben Sie zunächst einen Algorithmus, der den kürzesten Weg von jeder Ecke zu jeder anderen Ecke ermitteln kann. Dies wird als das "all pairs shortest path problem" bezeichnet.
5. Mit der Breadth First Suche das Irrgartenprogramm aus dem Rekursionskapitel überarbeiten, um den kürzesten Weg aus einem Irrgarten zu finden.
6. Schreiben Sie ein Programm zur Lösung des folgenden Problems: Sie haben zwei Krüge, einen 4-Gallonen- und einen 3-Gallonen-Krug. Keiner der beiden Krüge hat Markierungen auf ihnen. Es gibt eine Pumpe, mit der die Krüge mit Wasser gefüllt werden können. Wie kann man genau zwei Gallonen Wasser in den 4-Gallonen-Krug bekommen?
7. Verallgemeinern Sie das obige Problem, so dass die Parameter für Ihre Lösung die Größen der einzelnen Krüge und die endgültige Wassermenge, die im größeren Krug verbleibt, umfassen.
8. Schreiben Sie ein Programm, das folgendes Problem löst: Drei Missionare und drei Kannibalen kommen zu einem Fluss und finden ein Boot, in dem zwei Personen Platz haben. Alle müssen über den Fluss gelangen, um die Reise fortzusetzen. Wenn die Kannibalen jedoch jemals die Zahl der Missionare an beiden Ufern übersteigen, werden die Missionare aufgefressen. Finden Sie eine Reihe von Übergängen, die alle sicher auf die andere Seite des Flusses bringen.